In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os; 
os.path.expandvars('$CODE_MEMORY_ERRORS')

In [ ]:
import pingouin

In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
import pandas as pd
import seaborn as sns; print(sns.__version__)
from datetime import datetime
import pingouin as pg
data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))

print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

### prep

In [ ]:
from config2 import path_data
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from behav_proc import *

import numpy as np
#from collections import OrderedDict as odict
from config2 import envcode2env
from pingouin import ttest

from base2 import radius_cursor
radius_cursor
ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )


target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * (np.pi / 180)
target_angs / np.pi * 180

import behav_proc
from behav_proc import addBehavCols2, truncateNIHDfFromES, calcESthr

In [ ]:
%%javascript
// needed to access notebook path later
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

In [ ]:
import os

def getAddInfo():
    import datetime
    # Get the current date and time in the format YYYY-MM-DD HH:MM:SS
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Get the name of the current jupyter notebook
    #notebook_name = os.path.basename(os.environ.get("NOTEBOOK_PATH", ""))
    notebook_name = NOTEBOOK_FULL_PATH
    #print(notebook_name)
    add_ttle = f"{now}\n{notebook_name}"
    return add_ttle

def addTitleInfo(axs):
    add_ttle = getAddInfo()
    # Add a linebreak and the date, time, and notebook name to the ylabel of each axis
    if axs is not None:
        if isinstance(axs, (list, np.ndarray)): # If axs is a list or an array of axes
            for ax in axs:
                title = ax.get_title()
                new_title = f"{title}\n{add_ttle}"
                ax.set_title(new_title)
        else: # If axs is a single axis
            title = axs.get_title()
            new_title = f"{title}\n{add_ttle}"
            axs.set_title(new_title)
    else:
        title = plt.gcf()._suptitle.get_text()
        new_title = f"{title}\n{add_ttle}"
        plt.suptitle(new_title)

In [ ]:
print("NOTEBOOK_FULL_PATH:\n", NOTEBOOK_FULL_PATH)

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz_.pkl.zip')
#fnf = '/home/demitau/ownCloud/Current/merr_data/df_all_multi_tsz.pkl.zip'
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
#dfc_multi_tsz = pd.   read_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'))
print(df_all_multi_tsz['trial_group_col_calc'].unique())
print(df_all_multi_tsz['retention_factor_s'].unique())
# df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
#                             ' and retention_factor_s == "1.000"').copy()
df_all_multi_tsz_orig = df_all_multi_tsz

In [ ]:
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_witgt = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwtgt_we" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_orig = df

#%debug
df_witgt,dfall_witgt,ES_thr_witgt,_,_ = addBehavCols2(df_witgt.copy());
df,dfall,ES_thr,envv,pert = addBehavCols2(df);

df_wthr = df.copy()

In [ ]:
# to get retention from fits
loadBC = 0
if loadBC:
    bestCalcs = pd.read_pickle(pjoin(path_data,'herz_param_calced6.pkl.zip')
        ,compression='zip')

    for ute in [True,False]:
        bc = bestCalcs.query('use_true_errors == @ute').\
            groupby(['runname'])['fun'].mean().to_frame()
        i = bc['fun'].idxmin()

        rn = bc.loc[i].name

        alphas = bestCalcs.query(f'runname == "{rn}"')['alpha']
        print(f'{rn}\n  {alphas.mean():.2f} {alphas.std():.2f}' )

        #plt.hist(alphas)

In [ ]:
#df[['target_locs','org_feedback']].describe()
print( df_all_multi_tsz['trial_group_col_calc'].unique() )

checkErrBounds(df,['error','prev_error','error_deg'])

assert dfall.query('env == "stable"').trialwpertstage_wb.max() < 60
#qs0_fig3 = ' and trialwpertstage_wb > 0'

In [ ]:
display(df['error_deg'].describe())

In [ ]:
#df_wthr['err_sens'].max()

In [ ]:
# prepare for generalization analysis
import behav_proc
stds = behav_proc._calcStds(df)
stds = stds.to_frame().reset_index().rename(columns={'error_deg':'error_deg_initstd'}) 

print( df_all_multi_tsz.trial_group_col_calc.unique() )

df_all_multi_tsz_whtr = df_all_multi_tsz.merge(stds, on='subject')
df_all_multi_tsz_whtr_ = df_all_multi_tsz_whtr.\
    query('trial_group_col_calc in ["trialwe", "trialwtgt_we"]').copy()

df_all_multi_tsz_whtr_['env'] = df_all_multi_tsz_whtr_['environment'].apply(lambda x: envcode2env[x])

# def f(x):    
#     if x > np.pi:
#         x -= 2*np.pi
#     elif x < -np.pi:
#         x += 2*np.pi
#     return x
#df_all_multi_tsz_whtr_['error'] = df_all_multi_tsz_whtr_['error'].apply(f)
df_all_multi_tsz_whtr_['error_deg'] = df_all_multi_tsz_whtr_['error'] / np.pi * 180 
df_all_multi_tsz_whtr_['error_initstd'] = df_all_multi_tsz_whtr_.error_deg_initstd /  180 * np.pi 

print('trial_group_col_calc uniue = ', df_all_multi_tsz['trial_group_col_calc'].unique() )


checkErrBounds(df,['error','prev_error','error_deg'])
checkErrBounds(df_all_multi_tsz_whtr_,['error','prev_error','error_deg'])

shiftszs = df_all_multi_tsz_whtr_['trial_shift_size'].unique()
print(shiftszs)

_cols = [col for col in df_all_multi_tsz.columns if col.find('like') >= 0]
print(_cols)

#df_all_multi_tsz.query('trials <= 4 and subject == @subjects[0] and  trial_group_col_calc == "trialwe"')\
#    [['trials','trial_shift_size','error','prev_error']]

# truncate multi-ver ES
# remove trials with error > std_mult * std of error
dfall_notclean = df_all_multi_tsz_whtr_.query('err_sens.abs() <= @ES_thr')

from behav_proc import truncateDf
dfall_mshsz = truncateDf(dfall_notclean, 'err_sens', q=0.0, infnan_handling='discard', 
       cols_uniqify = ['subject','environment','trial_shift_size',
                       'trial_group_col_calc','retention_factor_s'])


In [ ]:
%matplotlib inline
sns.set(font_scale=1.3)

## No savings

In [ ]:
from behav_proc import checkSavingsNIH
checkSavingsNIH(dfall)

In [ ]:
corrs_per_subj_me_,corrs_per_subj  = corrMean(dfall, 
                stagecol = 'pert_stage', coln='err_sens', method='spearman')

In [ ]:
# show stat signif
stage_pairs = [(1,6),(3,8)]
ttrs2 = []
for s1,s2 in stage_pairs: 
    lst = [s1,s2]
    ttrs_sig, ttr = comparePairs(corrs_per_subj.reset_index().query('pert_stage.isin(@lst)'), 'r', 'pert_stage',
                                 paired=True)
    ttr['stage_pair'] = f'{s1}-{s2}'
    ttrs2 += [ttr]
ttrs2 = pd.concat(ttrs2)
display( ttrs2.query('pval <= 1e-2') )

In [ ]:
stage_pairs_nice = {"1-6":'first and last', "3-8":'second and third'}

display(ttrs2)
print('\n\nNo savings:')
for irow,row in ttrs2.query('alternative == "two-sided"').iterrows():
    sp = row['stage_pair']
    pv=row['pval']
    T=row['T']

    #print(sp,pv)
    print('ES during {} perturbations are not significantly different, t={:.2f}, p-value = {:.2e}.'.\
              format(stage_pairs_nice[sp],T,pv) )

In [ ]:
# show stat signif
stage_pairs = [(1,3),(6,8)]
print(stage_pairs)
ttrs2 = []
for s1,s2 in stage_pairs: 
    lst = [s1,s2]
    ttrs_sig, ttr = comparePairs(corrs_per_subj.reset_index().query('pert_stage.isin(@lst)'), 'r', 'pert_stage',
                                 paired=True)
    ttr['stage_pair'] = f'{s1}-{s2}'
    ttrs2 += [ttr]
ttrs2 = pd.concat(ttrs2).query('alternative != "two-sided"')
display( ttrs2.query('pval <= 1e-2')[['pval','ttstr']] )#

# Fig 2 CD

In [ ]:
## plots

In [ ]:
#me.subject.nunique()

In [ ]:
from figure import renameTickLabels, palette_stabrand

thr = "mestd*0"
me = dfall.groupby(['thr','subject','env'], observed=True).\
    mean(numeric_only=1).reset_index()
me_env = me
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), 
                 kind='violin', y='err_sens', 
    x='env', col='thr', order=['stable','random'],
                palette = palette_stabrand)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
from figure.mystatann import plotSigAll
ylast, ttrssig_env = plotSigAll(ax, 0.83, 0.05, ticklen=0.02,
       df=me, coln='err_sens', colpair = 'env')
    
ax.annotate('C', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')
fg.set_ylabels('Error sensitivity')
ax.set_xlabel('Environment')

fign = 'Fig2C'
plt.savefig(pjoin(path_fig,fign + '.svg'))
plt.savefig(pjoin(path_fig,fign + '.pdf'))
plt.show()
    
###################
# non-pooled stages and env comparison
me = dfall.groupby(['thr','subject','ps2_'], observed=True).\
    mean(numeric_only=1).reset_index()
me_ps = me
#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), kind='violin', y='err_sens', 
    x='ps2_', col='thr', order=['pre','pert','washout','rnd'],
                hue = 'ps2_')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
#addTitleInfo(fg.axes.flatten()[0])
#plt.gcf().add_subplot_labels(['C'])

ylast, ttrssig_ps = plotSigAll(ax, 2.05, 0.14, ticklen=0.05,
       df=me, coln='err_sens', colpair = 'ps2_', fontsize = 10)

ax.annotate('D', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

ax.set_xlabel('Experiment stage')
#ax.set_xticklabels(['No perturbation','Perturbation','Washout','Random'], 
#                   rotation=30)
ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )
renameTickLabels(ax, ps_2nice, rotation=30)

fg.set_ylabels('Error sensitivity')
fign = 'Fig2D'
plt.savefig(pjoin(path_fig,fign + '.svg'))
plt.savefig(pjoin(path_fig,fign + '.pdf'))

#fg.set_titles()

In [ ]:
## stats

In [ ]:
# to put in results
ttrs = []
for env in me_env.env.unique():
    r = compare0(me_env.query('env == @env'),'err_sens',cols_addstat=['err_sens'])
    r['env'] = env
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05').set_index('env')
assert len(ttrs_pos) == 2
display(ttrs_pos)

s0 = 'Averages of ES within participant. \n'
for env in me_env.env.unique():
    row = ttrs_pos.loc[env]
    s = (f"{env} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

display(ttrssig_env)
row = ttrssig_env.iloc[0]

s = f"Stable > random T={row['T']:.2f}, p-value={row['pval']:.2e}"
print(s0)
print(s)

In [ ]:
#comparePairs?

In [ ]:
# to put in results
#cocoln = 'ps2_'
ttrs = []
for ps in me_ps.ps2_.unique():
    r = compare0(me_ps.query('ps2_ == @ps'),'err_sens',cols_addstat=['err_sens'])
    r['ps2_'] = ps
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05')
assert len(ttrs_pos) == 4
display(ttrs_pos)
ttrs_pos = ttrs_pos.set_index('ps2_')

s0 = 'Averages of ES within participant: \n'
for ps in me_ps.ps2_.unique():
    row = ttrs_pos.loc[ps]
    s = (f"{ps} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

#display(ttrssig_ps)
#row = ttrssig_ps.iloc[0]

ttrssig_ps,_ = comparePairs(me_ps, 'err_sens', 'ps2_', alt=['greater'], 
                            paired=True, updiag=False)
display(ttrssig_ps[ttrssig_ps.columns[-5:]])
s1 = ''
for i,row in ttrssig_ps.iterrows():
    ttstr = row['ttstr']
    s = ttstr.replace(row['val2'], ps_2nice[row['val2']] ).replace(row['val1'],
                                                                   ps_2nice[row['val1']] )    
    s += f" T={row['T']:.2f}, p-value={row['pval']:.2e}; \n"
    s1 += s
#s = f"Stable > random p-value={row['pval']:.2e}"
print(s0)
print(s1)

In [ ]:
#me_ps.query('ps2_.isin(@psvals)')

In [ ]:
me_ps.ps2_.unique()

In [ ]:
# show that dif between pert and rand is not there
psvals = ["pert","rnd"]
ttrssig,ttrssig_all = comparePairs(me_ps.query('ps2_.isin(@psvals)'),
                         'err_sens', 'ps2_', alt=['two-sided'], 
                        paired=True, updiag=True, pooled=0)
display(ttrssig_all)
assert len(ttrssig_all) == 1
row = ttrssig_all.iloc[0]
if row['pval'] > 0.05:    
    s = 'The difference between ES in '
    s += '{} and {} conditions is not significant ('.format( 
        ps_2nice[row['val2']], ps_2nice[row['val1'] ] )
    s += f" T={row['T']:.2f}, p-value={row['pval']:.2e})."
    #s1 += s
    print(s)
else:
    print('Actually there is significance')

# Fig 2 AB

In [ ]:
tr = np.arange(len(pert))
c = 1 / np.pi  * 180
thrs = ["mestd*0"]
qs = 'thr in @thrs'
for lablet,varn,pertc in [('A', 'error_pscadj', 1.), 
                   ('B', 'err_sens', 0.06)]:
    #varn = 'org_feedback'
    #for varn in     
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    #for truncs, dfall in zip(['truncate q=5%'],[df_]):
    varn_eff = varn
    if varn.startswith('org_feedback'):
        dfall['vtp'] = np.array(dfall[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        dfall['vtp'] = np.array(dfall[varn]) * c 
        varn_eff = 'vtp'
    #elif varn.startswith('err_sens'):
    #    pert_shift = 
        #df['vtp'] = np.array(df[varn]) 
        #varn_eff = 'vtp'

    plt.figure(figsize=(12,4))
    fg = sns.relplot(data=dfall.query(qs),
        kind='line', x="trials", y=varn_eff, 
        col='thr', estimator='mean', errorbar='sd', height = 4, aspect=2.5)

    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--')
        ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn )
    if varn.startswith('error_'):
        fg.set_ylabels('Angular error [deg]')
        fg.set_titles('Error dynamics')
    else:
        fg.set_ylabels('Error sensitivity')
        fg.set_titles('Error sensitivity dynamics')
    fg.set_xlabels('Trial number')
    
    fign = 'Fig2'+lablet
    plt.savefig(pjoin(path_fig,fign + '.svg'))
    plt.savefig(pjoin(path_fig,fign + '.pdf'))

    
     
    #addTitleInfo(fg.axes.flatten()[0])
    ax.annotate(lablet, xy=(0, 1), xytext=(-60, 20), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

In [ ]:
#   debug large ES
# dfall.query('err_sens.abs() > 10')\
#     [['subject','trials','error','err_sens','prev_error']]

# t1,t2 = 20,49
# qs = 'trials <= @t2 and trials >= @t1'
# fg = sns.relplot(data=dfall.query(qs),
#     kind='line', x="trialwe", y='err_sens', 
#     estimator='mean', errorbar='sd', height = 4, aspect=2.5)
# fg.refline(x=24)

# df_all_multi_tsz['trial_group_col_calc'].unique()

### within target

In [ ]:
from figure.plots import relplot_multi
df2 = df_all_multi_tsz.\
    query('trial_shift_size == 1 and '
          'trial_group_col_calc.isin(["trialwtgt_we","trialwe"]) '
            ' and retention_factor_s == "1.000"').copy()

dfni = df2.loc[~np.isinf(df2['err_sens'])]
dfni_g = dfni.query('err_sens.abs() <= @ES_thr')
nremoved_pooled = len(dfni) - len(dfni_g)

sz = dfni.groupby(['subject'],observed=True).size()
sz_g = dfni_g.groupby(['subject'],observed=True).size()
mpct = ((sz - sz_g) / sz).mean() * 100
print(f'Mean percentage of removed trials = {mpct:.2f}%, '
      f'pooled = {nremoved_pooled / len(dfni) * 100:.2f}%')

dfall2 = truncateDf(dfni_g, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','env','trial_group_col_calc'])

In [ ]:
fg,df= relplot_multi(data=dfall2,
              ys=[['error_pscadj'],
                 ['err_sens']], x='trials',
                height=4,aspect=2, kind='line',
                     col = 'trial_group_col_calc',
                    estimator='mean', errorbar='sd')
fg.refline(y=0)
#r[0].refline(x=526)

# Corr with abs error (and prev) for Fig 3

In [ ]:
# correlation between ES and abs err within env
dfs = []
for varn in ['error','error_abs','prev_error','prev_error_abs']:
    def f(df_):
        #df_.query('err_sens < inf')
        r = pg.corr( df_[varn], df_['err_sens'],  method='spearman')
        return r
    corrs_abserr_per_subj = dfall.groupby(['thr','subject','env'],
                                         observed=True).apply(f)
    #display(corrs_abserr_per_subj)

    corrs_abserr = dfall.groupby(['thr','env'], observed=True).apply(f)
    #display(corrs_abserr)

    def f(df_):    
        a = df_.query('env == "stable"')['r']
        b = df_.query('env == "random"')['r']
        #print(len(a),len(b),a.abs().min(),b.abs().min())
        rs = []
        for alt in ['two-sided','greater','less']:
            #print(a.values, b.values)
            rs.append( pg.ttest( a, b, alternative = alt) )
        return pd.concat(rs)

    #%debug
    import warnings
    #warnings.filterwarnings("error")
    warnings.filterwarnings("ignore")
    #with warnings.catch_warnings(record=True) as w:
    ttest_of_corr_abserr = corrs_abserr_per_subj.\
        groupby(['thr']).apply(f)
    ttest_of_corr_abserr['varn'] = varn
    dfs += [ttest_of_corr_abserr]

dfr = pd.concat(dfs)
dfr = dfr.rename(columns={'p-val':'pval'})

corrs_abserr = corrs_abserr.rename(columns={'p-val':'pval'})

In [ ]:
#dfall.prev_error_abs

In [ ]:
print(getAddInfo())
thrs = [ "mestd*0"]
qs = 'thr in @thrs'
corrs_ = corrs_abserr.query(qs)
display( corrs_[corrs_['pval'] < 5e-2] )
with pd.option_context('display.max_rows', 500):
    dfr_ = dfr.query('varn == "prev_error_abs" and pval < 0.05')
    display(dfr_.query(qs))
    #display(dfr_.query('thr == "mestd*1.0"))

In [ ]:
pg.__version__

In [ ]:
# signed error
def f(df_):
    r = pg.corr( df_['error'], df_['err_sens'],  method='spearman')
    return r
corrs_err_per_subj = dfall.groupby(['thr','subject','env']).apply(f)

# preverror
def f(df_):
    r = pg.corr( df_['prev_error_abs'], df_['err_sens'],  method='spearman')
    return r
corrs_prevabserr_per_subj = dfall.groupby(['thr','subject','env']).apply(f)

def f(df_):    
    a = df_.query('env == "stable"')['r']
    b = df_.query('env == "random"')['r']
    rs = []
    for alt in ['two-sided','greater','less']:
        rs.append( pg.ttest( a, b, alternative = alt) )
    return pd.concat(rs)
ttest_of_corr_prevabserr = corrs_prevabserr_per_subj.groupby(['thr']).apply(f)
display(ttest_of_corr_prevabserr)

# Fig 3 (Within pert stage time resolved plots)

In [ ]:
def f(df_):
    #df_.query('err_sens < inf')
    r = np.cov( df_['err_sens'].values, df_['prev_error_abs'].values  )
    return r[0,1] / r[1,1]
covs_prevabserr_per_subj = dfall.groupby(['thr','subject','env']).apply(f)
covs_prevabserr_per_subj = covs_prevabserr_per_subj.to_frame().reset_index().rename(columns={0:'covabsprev'})

display(covs_prevabserr_per_subj)
dfall_aug = dfall.merge(covs_prevabserr_per_subj, on = ['thr','subject','env'])

dfall_aug = dfall_aug.assign(err_sens_prevabserrcorr = \
            dfall_aug['err_sens'] - dfall_aug['prev_error_abs'] * dfall_aug['covabsprev'])


In [ ]:
dfall = dfall.reset_index()

In [ ]:
dfc = dfall_aug.query('thr == "mestd*0"')
print(len(dfc))
dfc['trialwpertstage_wb'] = dfc['trialwpertstage_wb'].\
    where(dfc['environment'] == 0, dfc['trialwb'])
dfc['trialwpertstage_wb'] = dfc['trialwpertstage_wb'].astype(int)

assert not dfc.duplicated(['subject','trials']).any()
# nan-ify after pause
dfc.loc[dfc['trialwb'] == 0, 'err_sens'] = np.nan

In [ ]:
sns.__version__

## Fig 3

In [ ]:
# calc slopes
method = 'pearson'
corrs_per_subj_me_,_  = corrMean(dfall, stagecol = 'ps2_',
                                             method=method)
pcorrs_per_subj_me_,_ = corrMean(dfall, covar = 'prev_error_abs', 
                               stagecol = 'ps2_',
                                              method=method)

pswb2r = corrs_per_subj_me_.loc[('mestd*0',slice(None))].\
    to_dict()
print(pswb2r)
def f(row):
    #ps = row['pert_stage_wb']
    ps = row['ps2_']
    r = pswb2r['r'][ps]
    std_x = pswb2r['std_x'][ps]
    mean_x = pswb2r['mean_x'][ps]
    std_y = pswb2r['std_y'][ps]
    mean_y = pswb2r['mean_y'][ps]
    xs = row['trialwpertstage_wb']
    return mean_y + r * (xs - mean_x) / std_x * std_y 
dfc['pred'] = dfc.apply(f, axis=1)
#corrs_per_subj_me_ES['r'] * dfc['error_abs']

pswb2r = pcorrs_per_subj_me_.loc[('mestd*0',slice(None))].\
    to_dict()
print(pswb2r)
def f(row):
    #ps = row['pert_stage_wb']
    ps = row['ps2_']
    if ps == -1:
        return None
    r = pswb2r['r'][ps]
    std_x = pswb2r['std_x'][ps]
    mean_x = pswb2r['mean_x'][ps]
    std_y = pswb2r['std_y'][ps]
    mean_y = pswb2r['mean_y'][ps]
    #mean_z =  pswb2r['mean_z'][ps]
    xs = row['trialwpertstage_wb']
    return mean_y + r * (xs - mean_x) / std_x * std_y# + mean_z
dfc['ppred'] = dfc.apply(f, axis=1)

In [ ]:
pcorrs_per_subj.columns

In [ ]:
hue_order = dfc['pert_stage_wb'].unique()
col_order = ['pre', 'pert', 'washout', 'rnd']
hues = [[0],[1,3],[2,4],[5]]

# TODO: add fit
# TODO: start of pert
# TODO: sd instead of se
palette=['blue', 'orange', 'green', 'olive','cyan','brown']
df_ = dfc.query('trial_shift_size == 1')
fg = sns.relplot(data=df_, kind='line',
            x='trialwpertstage_wb', col='ps2_',
           y='err_sens', hue='pert_stage_wb',
                 ci = 'sd', palette = palette,
           facet_kws={'sharex':False},
                 hue_order=hue_order,
            col_order = col_order, legend=None)
# for ax in fg.axes.flatten():
#     ax.axhline(0,ls=':',c='red', alpha=0.7)
    
for i, ax in enumerate(fg.axes.flat):
    col_ = fg.col_names[i]
    ax.set_title(ps_2nice[ax.get_title()[7:]] )
    if col_ == 'rnd':
        continue
    sp = np.array(palette)[hues[i]]
    sp = list(sp)
    sns.lineplot(data=df_[df_['ps2_'] == col_], 
        x='trialwpertstage_wb', y='pred', 
        hue='pert_stage_wb', ax=ax, legend=None,
                palette = sp, dashes=[4,2])
    
    
#addTitleInfo(fg.axes.flat[0])
    
print(fg.hue_kws, fg.hue_names)
fg.refline(y=0, color='red')
#fg.map(plt.hist, 'tip').refline(0.15)
    
#fg.set_titles('{col_name}')
fg.set_xlabels('Trial number')
fg.set_ylabels('Error sensitivy')
    
lablet = 'A'
ax = fg.axes.flat[0]
ax.annotate(lablet, xy=(0, 1), xytext=(-60, 40), 
  fontsize=19, fontweight='bold', va='top', ha='left',
  xycoords='axes fraction', textcoords='offset points')
fnfig = pjoin(path_fig, f'Fig3_{lablet}_dynESpert_stage.pdf')
plt.savefig(fnfig)
plt.tight_layout()
plt.show()

###############################

#vn = 'err_sens_abserrcorr'
vn = 'err_sens_prevabserrcorr'
palette=['blue', 'orange', 'green', 'olive','cyan','brown']
df_ = dfc.query('trial_shift_size == 1')
fg = sns.relplot(data=df_, kind='line',
            x='trialwpertstage_wb', col='ps2_',
           y=vn, hue='pert_stage_wb',
                 ci = 'sd', palette = palette,
           facet_kws={'sharex':False},
                 hue_order=hue_order,
            col_order = col_order, legend=None)
# for ax in fg.axes.flatten():
#     ax.axhline(0,ls=':',c='red', alpha=0.7)
    
for i, ax in enumerate(fg.axes.flat):
    col_ = fg.col_names[i]
    ax.set_title(ps_2nice[ax.get_title()[7:]] )
    if col_ == 'rnd':
        continue
    sp = np.array(palette)[hues[i]]
    sp = list(sp)
    sns.lineplot(data=df_[df_['ps2_'] == col_], 
        x='trialwpertstage_wb', y='ppred', 
        hue='pert_stage_wb', ax=ax, legend=None,
                palette = sp, dashes=[4,2])        
    
#addTitleInfo(fg.axes.flat[0])
    
print(fg.hue_kws, fg.hue_names)
fg.refline(y=0, color='red')

#fg.set_titles('{col_name}')
fg.set_xlabels('Trial number')
fg.set_ylabels('Error sensitivy conditioned\non previous absolute error')


lablet = 'B'
ax = fg.axes.flat[0]
ax.annotate(lablet, xy=(0, 1), xytext=(-60, 40), 
  fontsize=19, fontweight='bold', va='top', ha='left',
  xycoords='axes fraction', textcoords='offset points')
fnfig = pjoin(path_fig, f'Fig3_{lablet}_dynESpert_stage.pdf')
plt.savefig(fnfig)
plt.tight_layout()
plt.show()

In [ ]:
#vn = 'err_sens_abserrcorr'
vn = 'err_sens_prevabserrcorr'
palette=['blue', 'orange', 'green', 'olive','cyan','brown']
df_ = dfc.query('trial_shift_size == 1')
fg = sns.relplot(data=df_, kind='line',
            x='trialwpertstage_wb', col='ps2_',
           y=vn, hue='pert_stage_wb',
                 ci = 'sd', palette = palette,
           facet_kws={'sharex':False},
                 hue_order=hue_order,
            col_order = col_order, legend=None)
# for ax in fg.axes.flatten():
#     ax.axhline(0,ls=':',c='red', alpha=0.7)
addTitleInfo(fg.axes.flat[0])
    
for i, ax in enumerate(fg.axes.flat):
    col_ = fg.col_names[i]
    if col_ == 'rnd':
        continue
    sp = np.array(palette)[hues[i]]
    sp = list(sp)
#     sns.lineplot(data=df_[df_['ps_'] == col_], 
#         x='trialwpertstage_wb', y='ppred', 
#         hue='pert_stage_wb', ax=ax, legend=None,
#                 palette = sp, dashes=[4,2])
print(fg.hue_kws, fg.hue_names)
fg.refline(y=0, color='red')

fg.set_titles('{col_name}')
fg.set_xlabels('Trial number')
fg.set_ylabels('Error sensitivy conditioned\non absolut error')

## Fig-3 related stats print

In [ ]:
method = 'spearman'
_,corrs_per_subj  = corrMean(dfall, stagecol = 'ps2_', method=method)
_,pcorrs_per_subj = corrMean(dfall, covar = 'prev_error_abs', 
        stagecol = 'ps2_', method=method)

# stats for Fig 3 caption
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r', cols_addstat=['r'])
    return ttrs

print(getAddInfo())

print('ps_-sep corr')
ttrs = corrs_per_subj.\
    groupby(['ps2_']).apply(f)
corrs_sig = ttrs.query('pval <= 0.05').reset_index()
cols = ['ps2_','dof','T','pval','alt','ttstr','r_mean', 'r_std']
display(corrs_sig[cols])

print('ps_-sep partial corr')
ttrs = pcorrs_per_subj.\
    groupby(['ps2_']).apply(f)
pcorrs_sig = ttrs.query('pval <= 0.05').reset_index()
display(pcorrs_sig[cols])

In [ ]:
corrmeth = corrs_per_subj['method'].unique()[0]
for n,sig in zip([f'{corrmeth} correlations between trial number and ES within subject',
                  ('Partial correlations between trial number and '
                  'ES within subject, conditioned on absolute error size')],
                 [corrs_sig, pcorrs_sig]):    
    s = n + ': '
    for irow,row in sig.iterrows():
        psn = ps_2nice[row['ps2_']]
        s += f"{psn} mean r = {row['r_mean']:.3f} (std={row['r_std']:.3f}) " +\
            f"r{row['ttstr'][-4:]} p-val {row['pval']:.2e}; "
    print(s[:-2] + '.')

In [ ]:
ps_2nice

# Variability (compare with Tan)

In [ ]:
import gc; gc.collect()

In [ ]:
histlens

In [ ]:
#dfc = dfall_aug.query('thr == "mestd*1.0"')
#dfc = dfall.query('thr == "mestd*1.0"')
# here we need all trials, w/o holes
#dfc = df_wthr.copy()
dfc = df.copy()

assert not dfc.duplicated(['subject','trials']).any()

## corr ES and variance (and other statistics), fixed histlen across subjects

from behav_proc import addWindowStatCols
dfcs,dfcs_fixhistlen,dfcs_fixhistlen_untrunc,histlens  = addWindowStatCols(dfc, ES_thr)

# this function just sets to NaNs large values
# me_pct_excl is info on what was excluded: mean_excl - mean percentage of excluded, std_excl - std percentage of excluded
dfcs_fixhistlen, me_pct_excl = behav_proc.truncLargeStats(dfcs_fixhistlen_untrunc, histlens, std_mult = 5.)

In [ ]:
dfcs_fixhistlen_untrunc.to_pickle(pjoin(path_data,'dfcs_fixhistlen_untrunc.pkl') )
dfcs_fixhistlen.to_pickle(pjoin(path_data,'dfcs_fixhistlen.pkl') )
me_pct_excl.to_pickle(pjoin(path_data,'me_pct_excl.pkl') )

In [ ]:
len(dfcs_fixhistlen_untrunc), len(dfcs_fixhistlen), len(dfall)

In [ ]:
dfcs_fixhistlen.groupby('subject').size().max(), dfcs_fixhistlen_untrunc.groupby('subject').size().max()

In [ ]:
dfcs_fixhistlen_untrunc['error_pscadj_mav3']

In [ ]:
mebad = me_pct_excl.query('not varn.str.startswith("error_pscadj_abs") and mean_excl >= 3')
display( mebad )

display(me_pct_excl['mean_excl'].describe([0.95]))

display(me_pct_excl.query('not varn.str.startswith("error_pscadj_abs")')['mean_excl'].describe([0.95]))


In [ ]:
# np.std?

# err = np.array([0.2195 , 0.2547]); 
# np.mean(err), np.std(err, ddof=1)

# dfcs0.query('subject_ind == 1 and trialwb <= 4').sort_values('trials')\
#     [['trials','trialwb','error_pscadj_abs','error_pscadj_abs_mav3','error_pscadj_abs_std3',
#       'error_pscadj_abs_Tan3','error_pscadj_abs_mav3','error_pscadj_abs_mavsq3',
#      'error_pscadj_abs_var3']]

# dfcs_fixhistlen.query('subject_ind == 1 and trialwb <= 4').sort_values('trials')\
#     [['trials','trialwb','error_pscadj_abs','error_pscadj_abs_mav3','error_pscadj_abs_std3',
#       'error_pscadj_abs_Tan3','error_pscadj_abs_mav3','error_pscadj_abs_mavsq3',
#      'error_pscadj_abs_var3']]

#### supp plots1

In [ ]:
gc.collect()
varnames_toshow0 = []
#for varn0 in ['error_pscadj_abs']:#'error_change']:
for varn0 in ['error_pscadj']:#'error_change']:
    for std_mavsz_ in range(2,30)[1::10]:
        varnames_toshow0_ = []
        for varn in  ['{varn0}_std{std_mavsz_}',
                      '{varn0}_invstd{std_mavsz_}',                     
                     '{varn0}_mavsq{std_mavsz_}',
                      '{varn0}_mav_d_std{std_mavsz_}',
                      '{varn0}_mav_d_var{std_mavsz_}',
                     '{varn0}_Tan{std_mavsz_}']:        
            varnames_toshow0_ += [varn.format(varn0=varn0,std_mavsz_=std_mavsz_)]
        varnames_toshow0 += [varnames_toshow0_[:3],varnames_toshow0_[3:]]
        
        
varnames_toshow0 = [['error_pscadj_std2','error_pscadj_std3',
                     'error_pscadj_std27'],
                     ['error_pscadj_invstd9','error_pscadj_invstd21',                     
                   'error_pscadj_invstd15', 'error_pscadj_Tan5']]

varnames_toshow = varnames_toshow0[0]
varnames_toshow

In [ ]:
gc.collect()

In [ ]:
# density plots (in place defintion)
def density_plot(x,y, **kwargs):
    #ax = sns.kdeplot(data=dfcs_fixhistlen,
    #           x='err_sens',y=vn, fill=False, hue=coln_col)
    df__ = pd.DataFrame( np.array(list(zip(x,y))))
    ax = sns.kdeplot(data=df__,
            x=x,y=y, fill=True, **kwargs)
    ax.axhline(0,ls=':', c='r')
    ax.axvline(0,ls=':', c='r')
    
    
for coln_col in ['env','ps_']:
    for varnames_toshow in varnames_toshow0: #[varnames_toshow0[:6],varnames_toshow0[6:]]:
        #dfcs_fixhistlen_untrunc
        for vn in varnames_toshow:
            g = sns.FacetGrid(col=coln_col, data=dfcs_fixhistlen)
            g.map(density_plot, "err_sens", vn, label='')                                                        
    break
    

In [ ]:
# density plots
ind = 0
for coln_col in ['env','ps_']:
    for varnames_toshow in varnames_toshow0:
        fg,df = relplot_multi(sep_ys_by = 'row', kind='density',             
                      data=dfcs_fixhistlen, x='err_sens',
                      ys=[varnames_toshow], col=coln_col, palette='Set1',
                     height = 5, facet_kws={'sharex':True, 'sharey':'row'})#, alpha=0.3)

        for ax in fg.axes.flatten():
            ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
            #ax.set_ylabel()
            ax.set_title(ttl)
        fg.refline(x=0)
        fg.refline(y=0)
        fg.fig.suptitle('Different statistics vs err sens')
        plt.tight_layout()     
        plt.savefig( pjoin(path_fig, f'ES2statDens_{varn0}_{coln_col}_{ind}.pdf') )
        plt.close()
        ind += 1
#    break

In [ ]:
# scatter plots, pooled
from figure.plots import relplot_multi
ind = 0
for coln_col in ['env','ps_']:
    for varnames_toshow in varnames_toshow0: #[varnames_toshow0[:6],varnames_toshow0[6:]]:
        fg,df = relplot_multi(sep_ys_by = 'row',             
              data=dfcs_fixhistlen, x='err_sens',
              ys=varnames_toshow, col=coln_col, palette='Set1',
             height = 5, alpha=0.3)
        for ax in fg.axes.flatten():
            ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
            ax.set_title(ttl)
        fg.refline(x=0)
        fg.refline(y=0)
        fg.fig.suptitle('Different statistics vs err sens')
        plt.tight_layout()     
        plt.savefig( pjoin(path_fig, f'ES2stat_{varn0}_{coln_col}_{ind}.pdf') )
        plt.close()
        ind += 1
    

In [ ]:
# mean over subjects
ind = 0
for coln_col in ['env','ps_']:
    for varnames_toshow in varnames_toshow0: #[varnames_toshow0[:6],varnames_toshow0[6:]]:        
        me = dfcs_fixhistlen.\
            groupby(['subject','env'], observed=True)\
            [['err_sens']+varnames_toshow].mean().reset_index()
        
        fg,df = relplot_multi(sep_ys_by = 'row', data=me, x='err_sens',
                              ys=varnames_toshow, col=coln_col,
                              height = 5, alpha=0.8, palette='Set1')
        
        for ax in fg.axes.flatten():
            ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
            ax.set_title(ttl)
        fg.refline(x=0)
        fg.refline(y=0)
        fg.fig.suptitle('Different statistics vs err sens')
        plt.tight_layout()     
        plt.savefig( pjoin(path_fig, f'ES2stat_mean_{varn0}_{coln_col}_{ind}.pdf') )
        plt.close()
        ind += 1
 

In [ ]:
dfcs_fixhistlen.duplicated(['subject','trials']).any()
dfcs_fixhistlen.groupby(['subject']).trials.diff().max()

In [ ]:
# dynamics across time plots
ind = 0
for varnames_toshow in varnames_toshow0: #[varnames_toshow0[:6],varnames_toshow0[6:]]:
    import gc;gc.collect()
    fg,df__ = relplot_multi(sep_ys_by = 'row', data=dfcs_fixhistlen, x='trials',
                          ys=varnames_toshow + ['err_sens'], hue='pert_stage', height = 5, kind='line',
                         palette='Paired')
    for ax in fg.axes.flatten():
        ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
        ax.set_title(ttl)
    fg.refline(x=0)
    fg.refline(y=0)
    fg.fig.suptitle('Different statistics vs err sens')
    plt.tight_layout()    
    plt.savefig( pjoin(path_fig, f'trial2stat_{varn0}_{ind}.pdf') )
    plt.close()
    ind += 1
    
    del df__

In [ ]:
gc.collect()
varnames_toshow = []
for varn0 in ['error_pscadj_abs']:#'error_change']:
    for std_mavsz_ in [2,3,4] + list(range(5,30) )[::3]:
        #varnames_toshow0_ = []
        for varn in  ['{varn0}_Tan{std_mavsz_}']:        
            varnames_toshow += [varn.format(varn0=varn0,std_mavsz_=std_mavsz_)]
        #varnames_toshow0 += [varnames_toshow0_[:3],varnames_toshow0_[3:]]
varnames_toshow        

In [ ]:
# plot Tan vesions
import gc;gc.collect()
ind = 0
# fg,df__ = relplot_multi(sep_ys_by = 'col', data=dfcs_fixhistlen, x='trials',
#                       ys=varnames_toshow + ['err_sens'], 
#                         hue='pert_stage', height = 5, kind='line',
#                      palette='Set1')
fg,df__ = relplot_multi(sep_ys_by = 'row', data=dfcs_fixhistlen, x='trials',
                          ys=varnames_toshow + ['err_sens'], hue='pert_stage', 
                        height = 5, kind='line',
                         palette='Paired')

for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Different statistics vs err sens')
plt.tight_layout()    
plt.savefig( pjoin(path_fig, f'trial2Tan_{varn0}_{ind}h.pdf') )
plt.close()
ind += 1

del df__

In [ ]:
# show that low history stds are related to erro change
sns.scatterplot(data=df_, x='error_pscadj_change',
               y='error_pscadj_std2')
plt.figure()
sns.scatterplot(data=df_, x='error_pscadj_change',
               y='error_pscadj_std3')

plt.figure()
sns.scatterplot(data=df_, x='error_pscadj_change',
               y='error_pscadj_std10')

#### supp plots2

In [ ]:
df__ = dfcs.query('thr == "mestd*1.0"')
df__['err_sens'].max()

In [ ]:
dfcs[dfcs['subject'].isin(subjects[:2])]

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings('ignore',category=FutureWarning)
#     sns.catplot(data=dfcs[dfcs['subject'].isin(subjects[:2])], 
#                 kind='scatter',
#                 y='err_sens', 
#             x=f'error_pscadj_std5', col='subject',
#             col_wrap=5, hue='env' )

In [ ]:
#dfcs_ = dfcs[dfcs['subject'].isin(subjects[:2])]
# one subj
for subj in subjects[:6]:
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        sns.pairplot(data=dfcs.query('subject == @subj'), 
                     vars=['err_sens',
                    f'error_pscadj_std5',
                  f'error_pscadj_invstd5' ], hue='env' )

In [ ]:
# all subj
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    sns.pairplot(data=dfcs, vars=['err_sens',f'error_pscadj_std5',
                              f'error_pscadj_invstd5' ], hue='env' )

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    sns.pairplot(data=dfcs.query('env == "stable"'), 
             vars=['err_sens',f'error_pscadj_std5',
                              f'error_pscadj_invstd5',
                               f'error_pscadj_mav5',
                   f'error_pscadj_mavsq5',
                   f'error_pscadj_mav_d_std5',
                   f'error_pscadj_mav_d_var5',
                  f'error_pscadj_Tan5' ],corner=1)

In [ ]:
import gc
gc.collect()

## Compute corrs

In [ ]:
from behav_proc import compare0
import pingouin as pg

In [ ]:
# compute corrs within subject
# env
import pingouin as pg
from joblib import Parallel, delayed
from itertools import product
import warnings

def run_corr(df_big, corr_method, cocoln, std_mavsz_, varn0, varn_suffix):
    def f(df_):
        with warnings.catch_warnings(record=True) as w:
            warnings.filterwarnings('ignore',category=UserWarning)
            warnings.filterwarnings('ignore',category=RuntimeWarning)

            r = pg.corr( df_[varn], df_['err_sens'],  method=corr_method)
        r['method'] = corr_method
        return r
    
    #dfcs_fixhistlen, cocoln, std_mavsz_, varn0, varn_suffix = args
    varn = f'{varn0}_{varn_suffix}{std_mavsz_}'
    #cols = ['thr','subject']
    cols = ['subject']
    if cocoln != 'None':
        cols += [cocoln]
    assert not df_big.duplicated(cols + ['trials']).any()
    corrs_per_subj = df_big.groupby(cols, 
        observed=True).apply(f, include_groups=False)
    corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})
    corrs_per_subj['std_mavsz'] = std_mavsz_
    corrs_per_subj['varn'] = varn
    corrs_per_subj['cocoln'] = cocoln
    
    print(cocoln, std_mavsz_, varn0, varn_suffix)
    return corrs_per_subj.reset_index()

# if use shepherd, it is rather slow even in parallel mode
#corr_methods = ['spearman','shepherd']
corr_methods = ['spearman']
cocols = ['None', 'env', 'ps2_']
#std_mavsz_range = range(2, maxhl+1, 3)
std_mavsz_range = histlens
varn0s = ['error_pscadj', 'error_pscadj_abs']
varn_suffixes = ['std', 'invstd', 'mavsq', 'mav_d_std', 'mav_d_var', 'Tan']
args = list(product(corr_methods, cocols, std_mavsz_range, varn0s, varn_suffixes))
print(len(args))

n_jobs = 40

ind = 0
if n_jobs > 1:
    # Execute in parallel
    #backend = 'multiprocessing' # 'loky'
    backend = 'loky' 
    prlcorr = Parallel(n_jobs=n_jobs, backend = backend)\
        (delayed(run_corr)( dfcs_fixhistlen,*arg ) for arg in args)
    
    cocoln2corrs_per_subj = pd.concat(prlcorr)
else:
    raise ValueError('I m lazy to implement')
    
#cocoln2corrs_per_subj = {}
#dfs = []
# for cocoln in ['None', 'env','ps_']:
#     for std_mavsz_ in range(2,30):
#         print(cocoln, std_mavsz_)
#         for varn0 in ['error_pscadj','error_pscadj_abs']:#'error_change']:
#             for varn in  [f'{varn0}_std{std_mavsz_}',
#                           f'{varn0}_invstd{std_mavsz_}',                     
#                          f'{varn0}_mavsq{std_mavsz_}',
#                           f'{varn0}_mav_d_std{std_mavsz_}',
#                           f'{varn0}_mav_d_var{std_mavsz_}',
#                          f'{varn0}_Tan{std_mavsz_}']:
#                 def f(df_):
#                     r = pg.corr( df_[varn], df_['err_sens'],  method='spearman')
#                     r['method'] = 'spearman'
#                     return r
#                 #cols = ['thr','subject']
#                 cols = ['subject']
#                 if cocoln != 'None':
#                     cols += [cocoln]
#                 assert not dfcs_fixhistlen.duplicated(cols + ['trials']).any()
#                 corrs_per_subj = dfcs_fixhistlen.groupby(cols, 
#                     observed=True).apply(f, include_groups=False)
#                 corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})
#                 corrs_per_subj['std_mavsz'] = std_mavsz_
#                 corrs_per_subj['varn'] = varn
#                 dfs += [corrs_per_subj.reset_index()]
#         #display(corrs_per_subj)
#     corrs_per_subj = pd.concat(dfs, ignore_index=1)
#     cocoln2corrs_per_subj[cocoln] =     corrs_per_subj 

# for cocoln in cocoln2corrs_per_subj:
#     cocoln2corrs_per_subj[cocoln]['cocoln'] = cocoln
#cocoln2corrs_per_subj_ = pd.concat(cocoln2corrs_per_subj.values())
gc.collect()

In [ ]:
# compute corrs after average across subjects (pretty useless acutally) 
# env
std_mavsz_range_ = list(range(3,6)) + list( range(6, maxhl+1, 3) )
dfs = []
cocoln2corrs_mesubj = {}
for cocoln in ['None', 'env','ps2_']:
    for std_mavsz_ in std_mavsz_range_:
        print(cocoln, std_mavsz_)
        for varn0 in ['error_pscadj','error_pscadj_abs']:#'error_change']:
            for varn in  [f'{varn0}_std{std_mavsz_}',
                          f'{varn0}_invstd{std_mavsz_}',                     
                         f'{varn0}_mavsq{std_mavsz_}',
                          f'{varn0}_mav_d_std{std_mavsz_}',
                          f'{varn0}_mav_d_var{std_mavsz_}',
                         f'{varn0}_Tan{std_mavsz_}']:
                def f(df_):
                    with warnings.catch_warnings(record=True) as w:
                        warnings.filterwarnings('ignore',category=UserWarning)
                        warnings.filterwarnings('ignore',category=RuntimeWarning)
                        r = pg.corr( df_[varn], df_['err_sens'],  method='spearman')
                    r['method'] = 'spearman'
                    return r
                #cols = ['thr','subject']
                cols_av = ['trials']
                cols = []
                if cocoln != 'None':
                    cols += [cocoln]
                cols_keep = cols_av + cols + ['err_sens',varn]
                dfcs_fixhistlen_me = dfcs_fixhistlen[cols_keep].\
                    groupby(cols_av + cols).mean().reset_index()
                assert not dfcs_fixhistlen_me.duplicated(cols + ['trials']).any()                
                if len(cols) == 0:
                    corrs_mesubj = f(dfcs_fixhistlen_me)
                else:
                    corrs_mesubj = dfcs_fixhistlen_me.groupby(cols, 
                        observed=True).apply(f, include_groups=False)
                corrs_mesubj = corrs_mesubj.rename(columns={'p-val':'pval'})
                corrs_mesubj['std_mavsz'] = std_mavsz_
                corrs_mesubj['varn'] = varn
                dfs += [corrs_mesubj.reset_index()]
        #display(corrs_mesubj)
    corrs_mesubj = pd.concat(dfs, ignore_index=1)
    cocoln2corrs_mesubj[cocoln] =     corrs_mesubj 

for cocoln in cocoln2corrs_mesubj:
    cocoln2corrs_mesubj[cocoln]['cocoln'] = cocoln
cocoln2corrs_mesubj_ = pd.concat(cocoln2corrs_mesubj.values())
gc.collect()

## Mixed effects

### read

In [ ]:
#sm.__version__

In [ ]:
dfcs_fixhistlen['environment'] = dfcs_fixhistlen['environment'].astype(int)
df_ = dfcs_fixhistlen[['environment','subject','trials','error_pscadj_abs_Tan29']]
assert not df_.duplicated(['subject','trials']).any()
#df_.to_csv('/home/demitau/dfcs.csv')

In [ ]:
del summary,result

In [ ]:
from datetime import datetime
import dateutil
#dtstart = dateutil.parser.parse('march 31 2024  18:10')
dtstart = dateutil.parser.parse('april 12 2024  01:00')

import glob
prls_fnf = glob.glob(path_data + '/prl_*_2024*')

prl = []
for p in prls_fnf:    
    #p=  pjoin(path_data, f'prl_{ind}.npz')
    if not os.path.exists(p):
        continue
    
    dt = datetime.fromtimestamp(os.path.getmtime(p))
    if not dt >= dtstart:
        continue    
    print(p, dt)
    prl_ = np.load(p,allow_pickle=1)['arr_0']
    print(len(prl_))
    for item in prl_:
        item['dt'] = dt
    #prl__ = [(dt,_item) for _item in prl_]
    prl += list(prl_)

print('Total len ',len(prl))

In [ ]:
del summ,fet,fet0,tab        
del summ,fet,summary
del res

In [ ]:
# READ: extract stuff from prl
import re
def cnv(s):
    v = np.nan
    if s != '':
        v = float(s)
    return v

#DEBUG = 1
DEBUG = 0
ind = 0 
mixmr = []
for ip,p in enumerate(prl):
#for ip,p in enumerate(prl[136:136+1]):        
    dd_ = p.copy()
    dd_['ind_in_prl'] = ip
    #del dd_['s']    
    
    varn = dd_['varn']
    s2summ = dd_['s2summary']
    
    for (s,scov),summ in s2summ.items():                
        if summ is None:
            dd_['converged'] = False
            continue        
        dd = {}
        dd.update(dd_)        
        dd = dd.copy()
        
        co = dd['cocoln']
        if co == 'env':
            dd['refcond'] = 'random'
        else:
            dd['refcond'] = 'pert'
        
        del dd['s2summary'] # because we need only one
            
        tab = summ.tables[0]
        converged  = tab.loc[4,3] == 'Yes'
        converged2 = tab.loc[5,3] == 'Yes'

        fet = summ.tables[1].rename(columns={'P>|z|':'pval', 'Coef.':'coef'} )    

        fet.pval = fet.pval.apply(cnv)
        fet.coef = fet.coef.apply(cnv)
        fet0 = fet.reset_index()
        dd['fet0'] = fet0
        
        def f(row):
            # Find all occurrences between [T. and ]
            s = row['index']
            match = re.findall(r"\[T\.([^\]]*)\]", s)
            if match:
                return match[0]
            else:
                return dd['refcond']
        fet0['cond'] = fet0.apply(f,1)

        d = fet[['pval','coef']].T.to_dict()
        dd['intercept_pval'] = summ.pvalues['Intercept'] #d['Intercept']['pval']
        dd['intercept_coef'] = summ.params['Intercept'] #d['Intercept']['coef']
                
        dd['varn_coef'] = summ.params[varn] #d[varn]['coef']
        dd['varn_pval'] = summ.pvalues[varn] #d[varn]['pval']
        dd['converged']  = converged
        dd['converged2'] = converged2
        dd['s'] = s
        dd['scov'] = scov
        dd['summary'] = summ
        
        dd['coefp_sig_max'] = np.nan
        dd['coefp_sig_min'] = np.nan
        if (s.find('*') < 0) and dd['varn_pval'] < 0.05:
            dd['coefp_sig_max'] = dd['varn_coef'] 
            dd['coefp_sig_min'] = dd['varn_coef'] 
            
        #row = mixmr.iloc[6]
        #fet0 = row['fet0']
        strs = fet0.query('index.str.contains(@varn) and pval <= 0.05')['cond'].values
        condssig = ','.join(strs)
        dd['condssig'] = condssig

        print(varn,s,scov,converged,converged2)
        if DEBUG:
            display(summ)
        if converged and s.find('*') >= 0:
            interactions = fet0.query('index.str.contains(":")').index.values
            fet0int = fet0.loc[interactions]
                                    
            fet0['coefp'] = np.nan
            fet0.loc[fet0int.index,'coefp'] =fet0.loc[fet0int.index,'coef'] + dd['varn_coef']
            fet0.loc[fet0.query('index == @varn').index,'coefp'] = dd['varn_coef']
            fet0int = fet0.loc[interactions] # for it to have coefp now
            
            
            
            fet0sig = fet0.query('pval <= 0.05')
            if len(fet0sig):
                dd['coefp_sig_max'] = fet0sig['coefp'].max()
                dd['coefp_sig_min'] = fet0sig['coefp'].min()
            dd['coefp_max'] = fet0['coefp'].max()
            dd['coefp_min'] = fet0['coefp'].min()

            rs = fet0int.query('pval <= 0.05')
            lsig = len( rs )            
            #lsigp = len( rs )
#             if dd['cocoln'] == 'env':
#                 num_sig_inter = lsig + int( cnv(d[varn]['pval']) <= 0.05 )

#                 #v1 = dd['varn_coef'] + fet0.loc[interactions].iloc[0].pval        \
#                 # compute effec t in the other environment
#                 v2 = dd['varn_coef'] + fet0.loc[interactions].iloc[0].coef        
#                 num_sig_inter_pos = 0
#                 num_sig_inter_pos += int( dd['varn_coef'] > 0 ) * int( dd['varn_pval'] < 0.05 )
#                 num_sig_inter_pos += int( v2 > 0) * (fet0.loc[interactions].iloc[0].pval < 0.05)
#                 dd['num_inter'] = 2
#             else:
            num_sig_inter = lsig + int( cnv(d[varn]['pval']) <= 0.05 )
                
            num_sig_inter_pos = int( dd['varn_coef'] > 0 ) * int( dd['varn_pval'] < 0.05 )
            for ir,r in rs.iterrows():
                coef_full = d[varn]['coef'] + float(r['coef'])
                num_sig_inter_pos += int(coef_full > 0)
            dd['coefp_nm_max'] = fet0int['coefp'].max()  # nm means not meain effect
            dd['coefp_nm_min'] = fet0int['coefp'].min()
            
            dd['num_inter'] = len(interactions) + 1                        
            dd['num_sig_inter'] = num_sig_inter
            dd['num_sig_inter_pos'] = num_sig_inter_pos
            
            
        
            if scov == "1" and DEBUG:
                raise ValueError('st')        
#        display(summ.wmess)
        del summ,fet,fet0,tab        
#             if scov != "1":
#                 display(summ)
#                 display(fet)
#                 raise Exception('f')
        

    #del dd_['res']

        mixmr += [dd.copy()]
        ind += 1
    #break
mixmr= pd.DataFrame(mixmr)
print('completed')

mixmr['inter'] = mixmr['s'].str.contains('\*')
mixmr['sshrt'] = mixmr['s'].str[10:]

def f(row):
    varn = row['varn']
    s = row['sshrt']
    if isinstance(s,str):
        s = s.replace(varn,'{varn}')
    return s
mixmr['sshrt'] = mixmr.apply(f,1)

print(len(prl),len(mixmr))

mixmr.loc[mixmr['num_inter'].isna(),'num_inter'] = 0
mixmr = mixmr.loc[~mixmr.s.str.contains('\* error_pscadj_abs')] # they are weird, because not prev

# take newer calcs
grp = mixmr.groupby(['cocoln','varn','sshrt','scov'])
mixmr = aggRows(mixmr,'dt','max', grp =grp )

def f(row):
    statn = row['varn'][len(row['varn0'])+1:]
    hl = row['histlen']
    hl_l = len(str(hl))
    statn = statn[:-hl_l]
    return statn
                  
mixmr['statn'] = mixmr.apply(f,1)

In [ ]:
save = 0
if save:    
    s = str(datetime.now())[:-7].replace(' ','_')
    mixmr.to_pickle(pjoin(path_data, f'mll_{s}.pkl.zip'), compression='zip')
    print('saved')

In [ ]:
#summ.params[varn]

In [ ]:
sz = mixmr.groupby(['cocoln','varn','sshrt','scov']).size()
print(sz.max())
#display(sz)
assert sz.max() == 1

In [ ]:
#mixmr = mixmr.drop_duplicates(subset= list(set(cols0) - set('ind_in_prl') ) )

In [ ]:
#mixmr.query('varn == "error_pscadj_Tan33" and cocoln == "ps2_"').sort_values(cols_fav)[cols0]#[cols_fav + ['sshrt']]

In [ ]:
#sz[sz==4]

In [ ]:
#mixmr.loc[112].summary.tables[1]

In [ ]:
#cols_exc = ['histlen','varn0','res','summary','varn_suffix', 's2summary','s']
cols_exc = ['histlen','varn0','res','summary','varn_suffix', 's2summary','converged',
            'converged2','s', 'excfmt','fet0']
cols0 = [c for c in mixmr.columns if c not in cols_exc]; cols0
#display(mixmr[cols])

cols_fav = ['cocoln', 'varn', 'scov', 
 'coefp_sig_max', 'coefp_sig_min', 'coefp_nm_max', 'coefp_nm_min', 
    'num_sig_inter_pos','sshrt','condssig'] #'num_inter',


In [ ]:
# cols = [c for c in mixmr.columns if c not in  ['res','varn','s2summary','s','summary']]
# with pd.option_context('display.max_rows', None):
#     display(mixmr.query('cocoln == "env" and inter == True and converged == True')[cols])

In [ ]:
gc.collect()

In [ ]:
qs = 'cocoln == "env" and varn_pval <= 0.05 and converged2 == True and excfmt.isnull()'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
    
qs += ' and coefp_sig_max > 0'
display(mixmr.sort_values('s').query(qs)[cols0])

### All positive slopes

In [ ]:
# with significant interactions with ALL positive slopes (so the most inconsistent slopes across conditions)
# there is only one for ps_ error_pscadj_abs_invstd7 and the slope size is pretty low
cols = [c for c in mixmr.columns if c not in cols_exc]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
    
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter_pos == num_inter'
ds_ = mixmr.sort_values(['inter','varn']).query(qs)
inds = ds_.index._data
display(ds_[cols])

ds_2 = mixmr.sort_values(['inter','varn']).query(qs + ' and scov != "1"')
inds2 = ds_2.index._data
print('and with random effects inds = ', inds)

In [ ]:
# 252 has pretty small slopes in pres and rnd, but large in rnd and pert
cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], row['scov'])    
    fet = row['summary'].tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    print(fet[cc])
    #break
varnames = list( mixmr.loc[inds]['varn'].values )
print(varnames)

In [ ]:
# plot relationship
h=4
me = dfcs_fixhistlen.\
    groupby(['subject','env'])[ ['err_sens']+varnames].mean().reset_index()
fg,df = relplot_multi(sep_ys_by = 'col', data=me, x='err_sens',ys=varnames, row='env',
                     height = h)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Mean within subject')

fg,df = relplot_multi(sep_ys_by = 'col', data=dfcs_fixhistlen, x='err_sens',ys=varnames, row='env',
                     height = h)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Pooled')

In [ ]:
# with at least 1 condition signif different (so with slopes not fully consistent)
cols = [c for c in mixmr.columns if c not in cols_exc + ['varn_pval'] ]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter > 1 and coefp_sig_max >= 0.01'
qs += ' and not s.str.contains("error_pscadj_abs")'
df_ = mixmr.sort_values(['cocoln','inter','varn']).query(qs)
inds = df_.index._data
#display(df_[cols])

df_[cols_fav]

### some positive slopes

In [ ]:
# with >= 1 significant slopes
# there is only one for ps_ error_pscadj_abs_invstd7 and the slope size is pretty low
cols = [c for c in mixmr.columns if c not in cols_exc]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
    
qs += ' and coefp_sig_max > 0 and inter == True and not s.str.contains("prev_")' 
ds_ = mixmr.sort_values(['inter','varn']).query(qs)
inds = ds_.index._data
display(ds_[cols])

ds_2 = mixmr.sort_values(['inter','varn']).query(qs + ' and scov != "1"')
inds2 = ds_2.index._data
print(f'and with random effects {len(inds2)} inds2 = ', inds)

In [ ]:
#[c for c in dfc.columns if c.find('error') >= 0]

In [ ]:
#mixmr.iloc[0]['s2summary'].keys()

In [ ]:
ds_2[cols_fav ]

In [ ]:
# the first one (with random effects for ps_) is error_pscadj_invstd19, it has rahter small slope (0.022)
# and it has a tendency (not signif) to decrease in random (-0.048)

# other indices (w/o random effects for env) are for std with high histlen 25+, 
# they have very high slope in random with strong (not signif) reduction in stable
# but it goes doown
cc = ['Coef.','P>|z|']
for ind in inds2:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    #display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet0 = row['fet0'][['index','cond','coef','coefp','pval']]#.tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    display(fet0)
    #break

###  with just one signif condition (so with slopes kinda consistent across conditions)

In [ ]:
# with just one signif condition (so with slopes kinda consistent across conditions)
cols = [c for c in mixmr.columns if c not in cols_exc + ['varn_pval'] ]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter == 1 '#'and coefp_sig_max >= 0.01'
qs += ' and not s.str.contains("error_pscadj_abs")'
df_ = mixmr.sort_values(['cocoln','inter','varn0','histlen']).query(qs)
inds = df_.index._data
print(len(inds))
varnames = mixmr.loc[inds,'varn'].values
#display(df_[cols])

#df_2 = mixmr.sort_values(['cocoln','inter','varn']).query(qs + ' and scov != "1"')
#inds = ds_2.index._data
#print(inds)

In [ ]:
#print(set(varnames) )
from collections import OrderedDict
#my_list = [1, 2, 3, 2, 5, 1]
list(OrderedDict.fromkeys(varnames))

In [ ]:
df_.columns

In [ ]:
#print(cols)
df_[cols_fav] #,'intercept_pval', 'intercept_coef',num_sig_inter

In [ ]:
# the first one (with random effects for ps_) is error_pscadj_invstd19, it has rahter small slope (0.022)
# and it has a tendency (not signif) to decrease in random (-0.048)

# other indices (w/o random effects for env) are for std with high histlen 25+, 
# they have very high slope in random with strong (not signif) reduction in stable
# but it goes doown
cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet = row['summary'].tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    display(fet)
    #break

In [ ]:
mixmr.varn.unique()

In [ ]:
from figure.plots import relplot_multi

In [ ]:
# scatter plots don't show a very clear relationship between std for high histlen and err_sens
fg,df__ = relplot_multi(sep_ys_by = 'row', data=dfcs_fixhistlen,x='err_sens',ys=varnames, col='env')
             #row='subject');
fg.refline(x=0)
fg.refline(y=0)

In [ ]:
# histograms of error_std for large histlen show that they have fat tail towards zero for many participants
dfcs_fixhistlen.query('subject_ind == 13').hist(varnames[-1])
dfcs_fixhistlen.hist(varnames[-1])

In [ ]:
display(mixmr.query('varn == @varnames[-1]')[cols0])
display(mixmr.loc[39].summary.tables[1])

### --- with more restriction on coefp

In [ ]:
# --- with more restriction on coefp
qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter == 1 and coefp_sig_max >= 0.1'
qs += ' and coefp_min > -0.3'
df_ = mixmr.sort_values(['cocoln','varn0','histlen','scov']).query(qs)
inds = df_.index.values

#cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['coefp_sig_min'])    
    fet0 = row['fet0']
    display(fet0.iloc[:-1][['index','coef','coefp','pval']])
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    #break
print(len(inds))

In [ ]:
# for small histlen (up to 4 inc)  std has large slope in stable (and small/moderate insignif diff in random)
# for large histlen (since 25) std has large slope for random (and moderate/large insignif diff in stable )

In [ ]:
varnames = mixmr.loc[inds,'varn'].values
print(varnames, set(varnames))
#varnames_toshow = #[varnames[0]] + [varnames[-1] ]

In [ ]:
relplot_multi(data=dfcs_fixhistlen,x='err_sens',ys=varnames_toshow[:1], col='env');
relplot_multi(data=dfcs_fixhistlen,x='err_sens',ys=varnames_toshow[1:], col='env');
             #row='subject');
#sns.relplot(kind='scatter',data=dfcs_fixhistlen, x=

### No inter env (for Tan comparison)

In [ ]:
# no inter
# for env all intercepts are significant
# for ps_ all non-random intercepts are significant
qs = 'varn_pval <= 0.05 and converged2 == True '
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == False and scov != "1" and cocoln == "env"'
# we exclude std3 because it is too close to std2 which is algebraically close to err sens formula
qs += ' and not s.str.contains("error_pscadj_abs") and varn != "error_pscadj_std3"'
qs += ' and coefp_sig_max >= 1e-3'
df_ = mixmr.sort_values(['cocoln','inter','varn0','histlen']).query(qs)
inds = df_.index._data
display(df_[cols_fav[:4] + ['intercept_coef','intercept_pval']])
print(len(inds))

varn_suffixes_nointer = df_['varn_suffix'].unique()
print(varn_suffixes_nointer)

In [ ]:
df_srt = df_.sort_values('varn_coef', ascending=False)[cols_fav + ['varn_pval']]
display(df_srt)

In [ ]:
#df_.iloc[0]['fet0']

In [ ]:
# TODO: apply to env for stable
def addcols(df, cocoln, cond, pref, prek_pattern):
    '''pref determines names of the output columns'''
    #prek = 'C(ps2_)[T.washout]'
    prek = prek_pattern.format(cocoln,cond)
    #prek = f'C({cocoln})[T.{cond}]'
    def f(row):
        #fet0 = row['fet0']
        varn_cur = row['varn']
        #k = prek + f':{varn_cur}'
        ps,pvs = row['summary'].params, row['summary'].pvalues
        
        k = prek
        if k not in ps:
            return None,None
        coef = ps[k]
        pv   = pvs[k]            
        icpt,icpt_pv = ps[k], pvs[k]

        return icpt,icpt_pv
    cols = []
    for s in ['icpt','icptpv']:
        cols += [pref + s]
    df[cols] = df.apply(f,1,result_type='expand')
    return cols
addcols(df_, 'env', 'stable', 's_', 'C({})[T.{}]')    
addcols(df_, 'env', 'stable', 'r_', 'Intercept')    
df_[cols_fav + ['s_icpt','s_icptpv','r_icpt','r_icptpv']]

In [ ]:
#row.keys()

In [ ]:
br = df_.query('r_icptpv < 0.05')#
#print('random intercept is signif')
#display(br[cols_fav + ['s_icpt','s_icptpv','r_icpt','r_icptpv']])
hlsig2,s_icpt2,s_icptpv2,r_icpt2,r_icptpv2 =\
    br.iloc[1][['histlen'] + ['s_icpt','s_icptpv','r_icpt','r_icptpv'] ]

br = df_.query('s_icptpv < 0.05 and r_icptpv < 0.05')#
print('When both intercepts are signif and different')
display(br[cols_fav + ['s_icpt','s_icptpv','r_icpt','r_icptpv']])
_,s_icpt,s_icptpv,r_icpt,r_icptpv =\
    br.iloc[0][['histlen'] + ['s_icpt','s_icptpv','r_icpt','r_icptpv'] ]
histlens_both_signif = br['histlen'].values
hlsig = histlens_both_signif
hlsig = ', '.join( map(str, hlsig ) )
print(histlens_both_signif)

rows = br.iloc[[0,-1]]
s0_icpt = ('With growing history length the slope estimate grows, while random and stable' 
      ' intercept values decrease: ')
for rowi,row in rows.iterrows():
# h1,h2 = histlens_both_signif[[0,-1]]
# c1,c2 = print(br['coefp_sig_max'].values[[0,-1]])
# r1,r2 = print(br['r_icpt'].values[[0,-1]])
# s1,s2 = print(br['s_icpt'].values[[0,-1]])
    s = ('for history length {}: slope {:.2f} (p-value = {:.2e}), ' 
    'random intercept = {:.2e} (p-value = {:.2e})'
    'stable intercept correction = {:.2e} (p-value = {:.2e}). ')\
        .format(row['histlen'], row['varn_coef'], row['varn_pval'], row['r_icpt'], row['r_icptpv'],
               row['s_icpt'],row['s_icptpv'])
    s0_icpt += s
print(s0_icpt)




In [ ]:
br = df_.nlargest(1,'varn_coef')
display(br[cols_fav + ['ind_in_prl']])#[['varn_coef','varn_pval']]
row = br.iloc[0]; print(row.name)
pv,coef, hlmax = row[['varn_pval','varn_coef','histlen']].values

br = df_.nsmallest(1,'varn_coef')
display(br[cols_fav + ['ind_in_prl']])#[['varn_coef','varn_pval']]
row = br.iloc[0]; print(row.name)
pv_min,coef_min, hlmin = row[['varn_pval','varn_coef','histlen']].values

In [ ]:
dfu = df_.groupby(['cocoln','statn'])['histlen'].unique()
print( dfu  )
lens =  dfu['env']['invstd'] 
mnl,mxl,diffl = min(lens),max(lens), np.diff(lens)
print(mnl,mxl,diffl)
lens = ', '.join( map(str, lens ) )
#between {mnl} and {mxl}. 
s = f'''We ran mixed linear models ES ~ error statistics
with two conditions (random,stable)
without interactions but with random effects. Here "error statistics" is one of 
many possible statistics of previous errors: mean, standard deviation, inverse standard deviation,
mean/standard deviation, mean squared/variance.
We selected those models that converged and have a significant positive slope.
We found that only models with 1 / standard deviation with history lengths {lens}.
In by publication by Tan at al [CITE] authors used mean squared/variance with history lengths 20. 
However for our data this statistics appears to be not the right one.
The maximum histlen we tried was 39 so it is possible that for longer lengths same holds, 
probably because standard deviation stops changing much. 
History length higher than 6 was increased in 3-trial step.
The largest slope {coef:.3f} (p-value = {pv:.2e}) was for history of length {hlmax}.
The smallest slope {coef_min:.3f} (p-value = {pv_min:.2e}) was for history of length {hlmin}.
The only history lengths for which intercepts for both conditions were significantly 
positive and different was {hlsig}.'''
s += s0_icpt
s += '''In all other cases intercepts were not significantly different from zero.
'''.replace('\n',' ').replace('  ',' ')
print('\n'+s)

# (intercept for random = {r_icpt:.3f}, it is > 0 
# with p-value {r_icptpv:.2e} and for stable it's difference from random was
# {s_icpt:.3f}, it is > 0 with p-value {s_icptpv:.2e}). 
# In addition for history length {hlsig2} the intercept {r_icpt:.3f} for random > 0 was significant 
# with p-value {r_icptpv:.2e}.

### No inter ps2_

In [ ]:
#dfall.query('subj == "01" and ps2_ == "washout" and block_name == "stable1"')

In [ ]:
# no inter
# for env all intercepts are significant
# for ps_ all non-random intercepts are significant
qs = 'varn_pval <= 0.05 and converged2 == True '
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == False and scov != "1" and cocoln == "ps2_"'
# we exclude std3 because it is too close to std2 which is algebraically close to err sens formula
qs += ' and not s.str.contains("error_pscadj_abs") and varn != "error_pscadj_std3"'
qs += ' and histlen <= 24'
qs += ' and coefp_sig_max >= 1e-05'
df_ = mixmr.sort_values(['cocoln','inter','varn0','histlen']).query(qs)
inds = df_.index._data
display(df_[cols_fav[:4] + ['intercept_coef','intercept_pval']])
print(len(inds))

varn_suffixes_nointer = df_['varn_suffix'].unique()
print(varn_suffixes_nointer)

In [ ]:
df_srt = df_.sort_values('varn_coef', ascending=False)[cols_fav + ['varn_pval']]
display(df_srt)

In [ ]:
br = df_.query('cocoln == "ps2_"').nlargest(1,'varn_coef')
display(br[cols_fav + ['ind_in_prl']])#[['varn_coef','varn_pval']]
row = br.iloc[0]
print(row.name)
pv,coef, hlmax = row['varn_pval'],row['varn_coef'],row['histlen']

In [ ]:
dfu = df_.groupby(['cocoln','statn'])['histlen'].unique()
print( dfu  )

lens = dfu['ps2_']['invstd']
mnl,mxl,diffl = min(lens),max(lens), np.diff(lens)
lens = ', '.join(map(str,lens))
print(mnl,mxl,diffl)

#between {mnl} and {mxl}. 
s = f'''We also ran mixed linear models for
four conditions (random, zero perturbation, perturbation, washout)
without interactions but with random effects. 
We selected those models that converged and have a significant positive slope.
Similar to the case with two conditions above, 
we found that only models with 1 / standard deviation with historly lengths {lens} 
satisfy these requirements.
This time the maximum histlen we used was 24 because it is the lenght of the shortest condition.
The largest slope {coef:.3f} (p-value = {pv:.2e}) was for history of length {hlmax}.
'''.replace('\n',' ').replace('  ',' ')
print('\n'+s)

In [ ]:
#for ind in inds:
for ind in [340]:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['coefp_sig_min'])    
    print(row['varn'])

    display(mixmr.loc[ind].summary.tables[1][['Coef.','P>|z|']])

### some slopes WITH interaction, only those stats that work for no interactions

In [ ]:
# with >= 1 significant slopes
# there is only one for ps_ error_pscadj_abs_invstd7 and the slope size is pretty low
cols = [c for c in mixmr.columns if c not in cols_exc]; cols

qs = 'converged2 == True and cocoln == "ps2_" '
qs += ' and coefp_sig_max > 0 and inter == True and not s.str.contains("prev_")' 
qs += ' and varn_suffix.isin(@varn_suffixes_nointer)'
# choose those that have more than one signif interaction (not necessarily positive)
qs += ' and condssig.str.contains(",")'  
qs += ' and histlen <= 24'

df_ = mixmr.sort_values(['cocoln','inter','varn_suffix','histlen']).query(qs + ' and scov != "1"')

display(df_[cols_fav])
inds = df_.index._data
print(f'and with random effects {len(inds)} inds = ', inds)

dftmp = df_.copy()
dftmp = dftmp.set_index('histlen', verify_integrity=True)

print( dftmp.index.values )

In [ ]:
# TODO: apply to env for stable
def addcols(df, cocoln, cond, pref):
    '''pref determines names of the output columns'''
    #prek = 'C(ps2_)[T.washout]'
    prek = f'C({cocoln})[T.{cond}]'
    def f(row):
        #fet0 = row['fet0']
        varn_cur = row['varn']
        k = prek + f':{varn_cur}'
        ps,pvs = row['summary'].params, row['summary'].pvalues
        if k not in ps:
            return None,None,None,None
        coef = ps[k]
        pv   = pvs[k]    

        k = prek
        icpt,icpt_pv = ps[k], pvs[k]

        return coef,pv,icpt,icpt_pv
    cols = []
    for s in ['coef','pv','icpt','icptpv']:
        cols += [pref + s]
    df[cols] = df.apply(f,1,result_type='expand')
    return cols
addcols(dftmp, 'ps2_', 'washout', 'w')    
dftmp[['varn','varn_coef','varn_pval','intercept_coef','intercept_pval'] + ['wcoef','wpv','wicpt','wicptpv']]

In [ ]:
rows = dftmp.reset_index().iloc[[0,-1]]
s0_slope = ('''with growing history length the slope estimate in perturbation grows and
the postive significant correction to it in washout as well
while random and stable intercept values decrease: ''').replace('\n',' ')
for rowi,row in rows.iterrows():
# h1,h2 = histlens_both_signif[[0,-1]]
# c1,c2 = print(br['coefp_sig_max'].values[[0,-1]])
# r1,r2 = print(br['r_icpt'].values[[0,-1]])
# s1,s2 = print(br['s_icpt'].values[[0,-1]])
    s = ('for history length {}: ' 
    'perturbation slope = {:.2e} (p-value = {:.2e}) '
    'washout slope correction = {:.2e} (p-value = {:.2e}). ')\
        .format(row['histlen'], row['varn_coef'], row['varn_pval'], row['wcoef'], row['wpv'])
    s0_slope += s
print(s0_slope)

In [ ]:
coef1,pv1 = dftmp.loc[12][['wcoef','wpv']].values
coef2,pv2 = dftmp.loc[15][['wcoef','wpv']].values

refcond = 'pert'
othercond = 'washout'
lens = ', '.join(map(str,df_.histlen.values))
s = f'''To check whether the slope changes across conditions, 
for those statistical measures we found above we also computed mixed linear model with interactions 
and random effects. We found that only two models had significant slope difference between some conditins:
for models with with history lengths = {lens}
where in {othercond} condition slopes were significantly different.
For all the models the intercept in this condition was not significantly different from the 
reference condition ({refcond}) but the slope was significantly higher than in reference condition. 
''' 
s = s.replace('\n',' ').replace('  ',' ') + '\n\nMore specifically, ' + s0_slope
s += ' This is consistent with naive visual inspection of Fig 2A'
s += ' suggesting that adaptation is washout is faster than in perturbation stage.'
print(s)

# For len 12: slope correction {coef1:.3f}, p-value = {pv1:.2e}, 
# for len 15: slope correction {coef2:.3f}, p-value {pv2:.2e}.

In [ ]:
cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    #display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet0 = row['fet0'][['index','cond','coef','coefp','pval']]#.tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    display(fet0)
    #break

## corr for all without regard for env/ps2_ (of ES and windows-stat measues)

In [ ]:
# mean across
ttrs_pos = cocoln2corrs_mesubj['None'].query('pval <= 0.05 and r > 0')
varnames = list(ttrs_pos.reset_index()['varn'].values)
#print(varnames )
print('For mean,then corr {} varnames are >0 correlated'.format(len(varnames) ) )

In [ ]:
# all trial together
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r', cols_addstat=['r'])
    return ttrs
m2vns={}

    # include groups is whether df that f receives as arg has grouping columns or not
ttrs = cocoln2corrs_per_subj.query('cocoln == "None"').\
    groupby(['varn','method'], observed=True).apply(f, include_groups=False)
print(getAddInfo())
#display(ttrs.query('pval <= 0.05 and ttstr == "r < 0"')\
#        [['pval','ttstr']])

ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')
#display(ttrs_pos[['pval','ttstr']])
for method in cocoln2corrs_per_subj.method.unique():
    varnames = ttrs_pos.loc[ttrs_pos.index.get_level_values('method') == method].reset_index()['varn'].values
    varnames = list(varnames)
    #varnames = list(ttrs_pos[(None,method)])
    #print(varnames )
    print(method, len(varnames), ' varnames are >0 correlated' )
    m2vns[method] = varnames

#print( ', '.join(varnames) )


#print ('Dif between methods ',set(m2vns['spearman']) ^ set(m2vns['shepherd']) )

In [ ]:
ttrs_pos = ttrs_pos.query('method == "spearman"')
ttrs_pos = ttrs_pos.sort_values(['pval'])
ttrs_pos[['pval','T','r_mean','r_std']]

In [ ]:
ttrs_pos = ttrs_pos.query('varn.str.contains("_invstd")').sort_values(['r_mean'], ascending=False)
ttrs_pos[['pval','T','r_mean','r_std']]

In [ ]:
#ttrs_pos.query('varn.str.endswith("std2")')

In [ ]:
#df_ = pd.DataFrame( {'varn':m2vns['spearman'] } )
#df_['']

# std2
# invstd 4-39
# abs invstd 2-39
# abs mav d var 4-39

m2vns['spearman']

In [ ]:
# plot ES vs windows stats
import warnings
varnames = list(ttrs_pos.reset_index()['varn'].values)
if len(varnames) <= 10:
    print(varnames)
    for env in ['stable','random']:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore',category=FutureWarning)
            me = dfcs_fixhistlen.query('env == @env').\
                groupby(['subject','env'])[['err_sens']+varnames].mean().reset_index()
            fg = sns.pairplot(data=me, 
                     y_vars = ['err_sens'], hue='env',
                         x_vars=['err_sens'] + varnames)#,corner=1)

    me = dfcs_fixhistlen.\
        groupby(['subject'])[['err_sens']+varnames].mean().reset_index()
    fg = sns.pairplot(data=me, 
             y_vars = ['err_sens'], 
                 x_vars=['err_sens'] + varnames)#,corner=1)
else:
    print('Too long list of varnames ')

In [ ]:
gc.collect()

## corr within env (of ES and windows-stat measues)

In [ ]:
# mesubj (useless)
# it gives 0 because for random it does not make sense to average across participants
# because randomness is not consistent across them
env2varnames = {}
ttrs = cocoln2corrs_mesubj['env']
for env in ['stable','random']:
    ttrs_pos = ttrs.query('pval <= 0.05 and r > 0 and'
                         ' env == @env')
#display(ttrs_pos[['pval','ttstr']])
    varnames = list(ttrs_pos.reset_index()['varn'].values)
    print(env,len(varnames))
    env2varnames[env] = varnames
varnames_isec = set(env2varnames['stable']) & set(env2varnames['random'])
varnames_isec = list(varnames_isec)
print('random ', set(env2varnames['random']))
print('isec varnames',varnames_isec  )

print('For mean, stable&random corr {} varnames are >0 correlated'.format(len(varnames) ) )

In [ ]:
# Find with windows stats are positive bot for random AND for stable
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs_ = compare0(df, 'r')
    return ttrs_
for method in cocoln2corrs_per_subj.method.unique():
    ttrs = cocoln2corrs_per_subj.query('cocoln == "env" and method == @method').\
        groupby(['varn','env'], observed=True).apply(f)
    print(getAddInfo())
    #display(ttrs.query('pval <= 0.05 and thr in @thrs')\
    #        [['pval','ttstr']])

    ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')
    print('All positive, both envs (separately)')
    #display(ttrs_pos[['pval','ttstr']])

    env2varnames = {}
    for env in ['stable','random']:
        ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0" and'
                             ' env == @env')
    #display(ttrs_pos[['pval','ttstr']])
        varnames = list(ttrs_pos.reset_index()['varn'].values)
        env2varnames[env] = varnames
    varnames_isec = set(env2varnames['stable']) & set(env2varnames['random'])
    varnames_isec = list(varnames_isec)
    print(method, 'isec varnames',varnames_isec  )
#print(varnames)


In [ ]:
# To be put into paper
mer = cocoln2corrs_per_subj.query('cocoln == "env"').\
    groupby(['varn','env'], observed=True)['r'].mean()

print(getAddInfo(),'\n')
r = ttrs.query('pval <= 0.05 and ttstr == "r > 0" '
    ' and varn.isin(@varnames_isec)').reset_index()[['varn','env','pval']].set_index(['varn','env'])
#d = r.T.to_dict()
#print(d)

for varn in varnames_isec:
    st = r.loc[(varn,'stable'),'pval']
    ra = r.loc[(varn,'random'),'pval']
    stabr = mer[(varn,'stable')]
    randr = mer[(varn,'random')]
    
    s = (f"{varn}: For random r>0 has p-val={ra:.2e} (mean r = {randr:.3f})"
        f" and for stable r>0 has p-val={st:.2e} (mean r = {stabr:.3f})")
    print(s)

In [ ]:
#me['env']

In [ ]:
# plot
h=4
varnames_toshow = varnames_isec[1:]
fg,df = relplot_multi(sep_ys_by = 'col', data=dfcs_fixhistlen, x='err_sens',ys=varnames_toshow, row='env',
                     height = h)
for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)

fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Pooled')
plt.tight_layout()

me = dfcs_fixhistlen.\
    groupby(['subject','env'])[['err_sens']+varnames_toshow].mean().reset_index()
fg,df = relplot_multi(sep_ys_by = 'col', data=me, x='err_sens',ys=varnames_toshow, row='env',
                     height = h)
for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Mean within subject')
plt.tight_layout()

In [ ]:
# fg = sns.relplot(data=dfcs_fixhistlen, col='subject',
#     y = 'err_sens', x=varnames_isec[0], row='env')

In [ ]:
from behav_proc import formatRecentStatVarnames
isec_nice = formatRecentStatVarnames(varnames_isec)
display(isec_nice)
print('; '.join(isec_nice))

In [ ]:
isec = set(env2varnames['stable']) & set(env2varnames['random'])
if len(isec):
    print('If we fix the history length first, then compute correlation '
      'within participant (separately for both environments)'
      ' and then choose variables for which r>0 stat significantly in both environments'
      ' simultaneously, we get ',', '.join(list(isec) ) )
else:
    print('We get nothing signif postive in both separately')

In [ ]:
#ttrs.query('varn == "error_pscadj_invstd10"')

In [ ]:
# plot
varnames = list(ttrs_pos.reset_index()['varn'].values)
print(varnames)
for env in ['stable','random']:
    plt.figure()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        fg = sns.pairplot(data=dfcs.query('env == @env').\
            groupby(['subject','env'])[['err_sens']+env2varnames[env]].mean().reset_index(), 
            y_vars = ['err_sens'], hue='env',
            x_vars=['err_sens'] + env2varnames[env])#,corner=1)

In [ ]:
# You are an expert in python pandas and data science.
# I have a pandas dataframe with the following columns
# ttrs_pos.columns = ['alt', 'val1', 'ttstr','varn','env']
# 'ttstr' can take two possible string values '>0' and '<0'. 
# 'env' can take two string values 'stable' and 'random'. 
# 'varn' can take several possible string values.
# I want to select rows that have ttstr = '>0' for both stable and random values of 'env'.
# How to do it?

## corr within ps2_ (of ES and windows-stat measues)

In [ ]:
# PS_
# all corrs are positive within participants
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r')
    return ttrs
ttrs = cocoln2corrs_per_subj.query('cocoln == "ps2_"').\
    groupby(['varn','ps2_'], observed=True).apply(f)
print(getAddInfo())
ttrssig = ttrs.query('pval <= 0.05')
#display(ttrssig[['pval','ttstr','T']])

ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')
display(ttrs_pos[['pval','ttstr']])
varnames = list(ttrs_pos.reset_index()['varn'].values)
print(varnames)
#loc[:,'error_pscadj_std5',:]

# ttrs = corrs_per_subj2.\
#     groupby(['thr','ps_']).apply(f)
# print(getAddInfo())
# display(ttrs.query('pval <= 0.05 and thr in @thrs'))

In [ ]:
ps_v = dfall.reset_index()['ps2_'].unique()
len(ps_v), ps_v

In [ ]:
# count number of significant phases
nu = ttrs_pos.reset_index().groupby(['varn'])[['ttstr','ps2_']].nunique()
nu = nu.reset_index()
display(nu[nu['ps2_'] > 2])

In [ ]:
varns_good = nu[nu['ps2_'] == len(ps_v)].varn.values
print(varns_good)

varns_pre_good = nu[nu['ps2_'] == len(ps_v)-1].varn.values
print(varns_pre_good)

from behav_proc import formatRecentStatVarnames
isec_nice = formatRecentStatVarnames(varns_good)
display(varns_good)
print('; '.join(isec_nice))

In [ ]:
ttrs_pos.query('varn.isin(@varns_pre_good)')

In [ ]:
ttrssig_g = ttrssig_pos.reset_index().query('varn in @varns_good')\
    [['varn','ps_','pval','ttstr']]
display(ttrssig_g)

In [ ]:
mer = cocoln2corrs_per_subj.query('cocoln == "ps2_"').\
    groupby(['varn','ps2_'], observed=True)['r'].mean()

for varn in varns_good:
    print('  ',varn)    
    
    r = ttrssig_g.query('varn == @varn').\
        reset_index()[['varn','ps2_','pval']]
    d = r.T.to_dict()
    #print(d)
    s = ''
    for k,v in d.items(): 
        ps_nice = ps_2nice[v['ps2_']].lower()
        fv = v['pval']
        s += f"for {ps_nice} "            
        s += f' mean r = {mer[(varn,v["ps2_"])]:.3f} and p-val={fv:.3e}; '        
            
        
    print(s)

## Histlen individ per subject

In [ ]:
# dfcs_multi
# good to add block name because we make a pause between so supposedly we loose memory about last errors
dfcs2 = dfc.sort_values(
    ['pert_seq_code', 'subject', 'trial_group_col_calc','trials'])
grp = dfcs2.\
    groupby(['pert_seq_code', 'subject', 'trial_group_col_calc','block_name'],
           observed=True)

vars_to_av = ['error_pscadj', 'error_pscadj_abs'] 

dfs = []
#for std_mavsz_ in range(2,4):
for std_mavsz_ in range(2,30):    
    print(std_mavsz_)
    df_ = dfcs2.copy()
    df_['histlen'] = std_mavsz_
    for varn in vars_to_av:
        for g,gi in grp.groups.items():
            df_.loc[gi,f'{varn}_std'] = df_.loc[gi,varn].shift(1).rolling(std_mavsz_).std()   
            df_.loc[gi,f'{varn}_mav'] = df_.loc[gi,varn].shift(1).rolling(std_mavsz_).mean()   
                
        df_[f'{varn}_invstd'] = 1/df_[f'{varn}_std']
        df_[f'{varn}_var']    = df_[f'{varn}_std'] ** 2
        df_[f'{varn}_mavsq']  = df_[f'{varn}_mav'] ** 2
        df_[f'{varn}_mav_d_std']  = df_[f'{varn}_mav'].abs() / df_[f'{varn}_std']
        df_[f'{varn}_mav_d_var']  = df_[f'{varn}_mav'].abs() / df_[f'{varn}_var']
        df_[f'{varn}_Tan']    = df_[f'{varn}_mavsq'] / df_[f'{varn}_var']
    dfs += [df_]        
dfcs0 = pd.concat(dfs, ignore_index=True)

# only chose points where prev error is not too small (to compare with ES later)
dfcs_multi  = truncateDf(dfcs0, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','histlen']) #'env','thr',

gc.collect()

In [ ]:
# compute corrs
from behav_proc import compare0
# env
# here dfcs_multi varnames don't contain histlen, it is a sep column
histlens = dfcs_multi['histlen'].unique()
dfs = []
import pingouin as pg
for cocoln in ['None', 'env']:#,'ps_']:
    #for std_mavsz_ in histlens:    
    for varn0 in ['error_pscadj','error_pscadj_abs']:
        for varn in  [f'{varn0}_std',
                      f'{varn0}_invstd',                     
                     f'{varn0}_mavsq',
                      f'{varn0}_mav_d_std',
                      f'{varn0}_mav_d_var',
                     f'{varn0}_Tan']:            
            def f(df_):
                r = pg.corr( df_[varn], df_['err_sens'],  method='spearman')
                r['method'] = 'spearman'
                return r
            cols = ['histlen','subject']
            if cocoln != 'None':
                cols += [cocoln]
            print('Starting ' ,cocoln, varn, cols)
            assert not dfcs_multi.duplicated(cols + ['trials']).any()
            corrs_per_subj = dfcs_multi.groupby(cols, observed=True).apply(f)
            corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})            
            corrs_per_subj['varn'] = varn                    
            corrs_per_subj['cocoln'] = cocoln #corrs_per_subj 
            dfs += [corrs_per_subj.reset_index()]
    
cocoln2corrs_per_subj_multi = pd.concat(dfs, ignore_index = True)

gc.collect()

In [ ]:
len(cocoln2corrs_per_subj_multi),len(cocoln2corrs_per_subj_.query('cocoln.isin(["None","env"])'))

In [ ]:
from behav_proc import formatRecentStatVarnames
def printNice(ttrs):
    for rowi,row in ttrs.iterrows():
        varn = row['varn']
        varn_nice = formatRecentStatVarnames([varn],'')[0]
        s = varn_nice
        s += f" (history length mean={row['histlen_mean']:.1f} trials, std={row['histlen_std']:.1f} trials)"
        print(s)

In [ ]:
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r', cols_addstat = ['histlen'])
    return ttrs

for cocoln in ['None', 'env']: #,'ps_']:
    # within env or not?
    print('###### cocoln = ',cocoln)
    
    # choose best p-value among different histlens
    if cocoln != 'None':
        cocoln_add = [cocoln]
    else:
        cocoln_add = []    
    df_ = cocoln2corrs_per_subj_multi.query('cocoln == @cocoln')
    cols0 = ['subject','varn']    
    cols0 += cocoln_add
    grp = df_.groupby(cols0, observed=True )
    
    # take with with the lower p-value among all histlens
    dfr = aggRows(df_, 'pval', 'min',  grp, coltake = 'corresp') 
    
    display('r info:', dfr.\
        groupby(cocoln_add + ['varn'], observed=True)['r'].describe() )
    display('(for r>0) histlen info:', 
        dfr.query('r > 0').groupby(cocoln_add + ['varn'], 
                                   observed=True)['histlen'].describe() )
    display('(for r>0) pval info:', 
        dfr.query('r > 0').groupby(cocoln_add + ['varn'], 
                                   observed=True)['pval'].describe() )
    # compare r with zero
    cols1 = ['varn']
    cols1 += cocoln_add
    ttrs = dfr.\
        groupby(cols1, observed=True).apply(f).reset_index()
    print(getAddInfo())
    
    cols = ['varn','pval','ttstr','histlen_mean','histlen_std']
    cols = cocoln_add + cols
    
    ttrs_neg= ttrs.query('pval <= 0.05 and ttstr == "r < 0"')        
    ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')   
    display(ttrs_neg[cols])
    display(ttrs_pos[cols])
    
    varnames = list(ttrs_pos.reset_index()['varn'].values)    
    if cocoln != 'None':
        varnames_ext = zip(varnames, list(ttrs_pos.reset_index()[cocoln].values) )
        varnames_ext  = list(varnames_ext)
        print('r>0 varnames ',varnames_ext)
    else:
        print('r>0 varnames' , ', '.join(varnames) )
    
    printNice(ttrs_pos)    

## Error consist vs stats (useless)

In [ ]:
dfcs2 = truncateDf(dfcs_fixhistlen, 'err_sens_change', 
       q=0.0, infnan_handling='discard', 
       cols_uniqify = ['subject','env'])

In [ ]:
for varn in varnames_toshow:
    coln_same = 'err_sign_same3'
    qs = f'{coln_same} != 0.'
    grp = dfcs2.query(qs).groupby(['subject',coln_same,'env'],
                                 observed=True)
    dfme = grp.mean(numeric_only=1).reset_index()
    dfme['Environment'] = dfme['env']

    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        warnings.filterwarnings('ignore',category=UserWarning)
        fg = sns.catplot(data=dfme, kind='box',
             y =varn, x=coln_same,
             hue='Environment',hue_order=['stable','random'],
             palette=['tab:orange','tab:grey'],
            showfliers=True, legend_out=False, height=4)
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',c='r')

## plot of var

In [ ]:
# plot VAR
use_var_pubnames = False
#%debug
for suff in ['std5','invstd5']:
    for varn, pertc in [('error_pscadj', 0.01), ('error_change',0.01), 
                        ('err_sens', 0.017)]:
        c = 1 / np.pi  * 180
        #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
        for truncs, df in zip(['mestd * 1.0'],[dfcs]):
            plt.figure(figsize=(12,4))
            #qs = 'trial_group_col_calc in ["trialwe","trialwtgt_we"]'
            #qs += ' and pert_seq_code == 0'
            #df_= df.query(qs)
            df_ = df
            varn_eff = f"{varn}_{suff}"    
            ax = sns.lineplot(x="trials", y=varn_eff,  estimator='mean', ci='sd',
                         data=df_)
            vshift = 0.
            if varn.startswith('err_sens'):
                vshift = 0.5  
            # important to put this legend call before plotting pert and env
            #plt.legend(title='', loc='lower right', labels=['within tgt', 'across tgt'])
            ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
            #ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
            #        alpha=0.2, label='environment')

            axlbl = varn
            if use_var_pubnames:
                axlbl = var_pubnames[varn]
            else:
                ax.set_title(f'{varn}  {truncs}')

            ax.axhline(0,ls=':',c='red', alpha=0.7)
            ax.set_ylabel(axlbl + f'_{suff}' )
            addTitleInfo(ax)


In [ ]:
mavsz = 10
fg = sns.relplot(kind='line', x="trials", 
                 y=f'err_sens_mav{mavsz}',  estimator='mean', ci='sd',
                 col='subject', col_wrap=4,
                data=df_)
for ax in fg.axes.flatten():
    ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
    ax.axhline(0,ls=':',c='red', alpha=0.7)
        
plt.suptitle(f'err_sens_mav {mavsz}')
addTitleInfo(None)

In [ ]:
# plot MAV
mavsz = 10
use_var_pubnames = False
#%debug
for varn, pertc in [('error_pscadj', 0.01), 
                    ('error_change',0.001),
                    ('err_sens', 0.017)]:
    c = 1 / np.pi  * 180
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    for truncs, df in zip(['mestd * 1.0'],[dfcs]):
        plt.figure(figsize=(12,4))
        df_ = df
        varn_eff = f"{varn}_mav{mavsz}"  
        ax = sns.lineplot(x="trials", y=varn_eff,  estimator='mean', ci='sd',
                     data=df_)

        vshift = 0.
        if varn.startswith('err_sens'):
            vshift = 0.5
  
        # important to put this legend call before plotting pert and env
        #plt.legend(title='', loc='lower right', labels=['within tgt', 'across tgt'])
        ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
        #ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
        #        alpha=0.2, label='environment')
        
        axlbl = varn
        if use_var_pubnames:
            axlbl = var_pubnames[varn]
        else:
            ax.set_title(f'{varn}  {truncs}')
            
        ax.axhline(0,ls=':',c='red', alpha=0.7)
        ax.set_ylabel(axlbl + '_mav' )
        addTitleInfo(ax)
        

In [ ]:

# #Modifying @jfb method, gives the function below which worked fine on ipykernel-5.3.4.
# from IPython.display import display, Javascript
# def getNotebookName():
#     display(Javascript('IPython.notebook.kernel.execute("NotebookName = " + "\'"+window.document.getElementById("notebook_name").innerHTML+"\'");'))
#     try:
#         _ = type(NotebookName)
#         return NotebookName
#     except:
#         return None
# getNotebookName()

In [ ]:
dfall.columns

In [ ]:
dfall.groupby('subject')['movement_duration'].mean().mean()

In [ ]:
dfall.groupby('subject')['trial_duration'].mean().mean()

In [ ]:
plt.get_su

In [ ]:

os.environ.get("NOTEBOOK_PATH")

In [ ]:
from 

# Error consist vs ES (Herzfeld test)

## Error change

In [ ]:
# truncateDf(df_wthr, 'err_sens', 
#        q=0.0, infnan_handling='discard', 
#        cols_uniqify = ['subject','env'], verbose=True);

In [ ]:
dfc = df_wthr.copy() # df_thr is untruncated and we need it to compute error change


dfc = dfc.query('err_sens.abs() <= @ES_thr')

dfcs_ = truncateDf(dfc, 'err_sens_change', 
       q=0.0, infnan_handling='discard', 
       cols_uniqify = ['subject','env'], verbose=True)

# make sure I have not changed the algo
assert len(dfcs_.query('err_sign_same == 0')) == 0

In [ ]:
behav_proc.getQueryPct(dfcs_,'err_sign_same == 0')
for i in range(2,4):
    behav_proc.getQueryPct(dfcs_,f'err_sign_same{i} == 0')

In [ ]:
#from figure.plots import relplot_multi

In [ ]:
ax.get_xticks()

In [ ]:
#dfme.query('env == @env').query('err_sign_same3 == -1')

In [ ]:
# Error change
import warnings
from figure.mystatann import plotSig0All, plotSig0
from figure import renameTickLabels
dfcs2  = truncateDf(dfcs_, 'err_sens_change', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env'])

#coln_same = 'err_sign_same3'
coln_same = 'err_sign_same'
qs = f'{coln_same} != 0.'
grp = dfcs2.query(qs).groupby(['subject',coln_same,'env'],
                             observed=True)
dfme = grp.mean(numeric_only=1).reset_index()
dfme['Environment'] = dfme['env']

with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    warnings.filterwarnings('ignore',category=UserWarning)
    fg = sns.catplot(data=dfme, kind='box',
         y ='err_sens_change', x=coln_same,
         hue='Environment',hue_order=['stable','random'],
         palette=['tab:orange','tab:grey'],
        showfliers=True, legend_out=False)
#fg.apply(lambda ax: ax.axline(y=0))
#def f(data, **kwargs):
#    plt.axhline(y=0)
#fg.map(f, ls=":", c=".5" )
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',c='r')
    
ttrs0,ttrssigs0 = [],[]
for env_,xt,xv in [('stable',-0.25,-1),('random',0.25,-1),
                  ('stable',0.75,1),('random',1., 1)]:
    #for xv in [-1,1]:
    for alt in ['greater','less','two-sided']:
        ttrs0_,ttrssig0 = plotSig0(ax, x=xv, y=0.63, df=dfme.query('env == @env_'), 
                 coln='err_sens_change', colx = coln_same, xt=xt, alt=alt)
        ttrs0_['env'] = env_
        ttrs0_[coln_same] = xv
        ttrs0 += [ttrs0_]
        if len(ttrssig0):
            row = ttrssig0.iloc[0]
            ttrssig0['env'] = env_
            ttrssig0[coln_same] = xv
            #print(env_,xv,row['pval'],ttstr)
            ttrssigs0 += [ttrssig0]
ttrssigs0 = pd.concat(ttrssigs0, ignore_index=1)
ttrs0 = pd.concat(ttrs0, ignore_index=1)
        
#plotSigAll(ax, 1, 0.1,  df=dfme, lab2tick = )
    
plt.suptitle('Error sensitivity dependence \non error sign consitency',y=1.1)
ax.set_ylabel('Error sensitivity change,\nmean within participant')
ax.set_xlabel('Sign consistency')
renameTickLabels(ax, {'-1':'inconsistent', '1':'consistent' })
plt.savefig(pjoin(path_fig, f'ESchange_per_sign_consist_{coln_same}.pdf'))

In [ ]:
dfcs2.groupby()['err_sens_change']

In [ ]:
dfme[]

In [ ]:
dfcs2  = truncateDf(dfcs_.query('trialwpertstage_wb <= 60'), 'err_sens_change', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env'])
fg = sns.relplot(data=dfcs2, kind='line',  x='trialwpertstage_wb',y='err_sens_change', hue='err_sign_same', col='env' )
#kind='line',
fg.refline(y=0)

In [ ]:
# Error change, but without initial trials in pert
dfcs2  = truncateDf(dfcs_, 'err_sens_change', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env']).query('trialwpertstage_wb != 1')

#coln_same = 'err_sign_same3'
coln_same = 'err_sign_same'
qs = f'{coln_same} != 0.'
grp = dfcs2.query(qs).groupby(['subject',coln_same,'env'],
                             observed=True)
dfme = grp.mean(numeric_only=1).reset_index()
dfme['Environment'] = dfme['env']

with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    warnings.filterwarnings('ignore',category=UserWarning)
    fg = sns.catplot(data=dfme, kind='box',
         y ='err_sens_change', x=coln_same,
         hue='Environment',hue_order=['stable','random'],
         palette=['tab:orange','tab:grey'],
        showfliers=True, legend_out=False)
#fg.apply(lambda ax: ax.axline(y=0))
#def f(data, **kwargs):
#    plt.axhline(y=0)
#fg.map(f, ls=":", c=".5" )
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',c='r')
    
ttrs0,ttrssigs0 = [],[]
for env_,xt,xv in [('stable',-0.25,-1),('random',0.25,-1),
                  ('stable',0.75,1),('random',1., 1)]:
    #for xv in [-1,1]:
    for alt in ['greater','less','two-sided']:
        ttrs0_,ttrssig0 = plotSig0(ax, x=xv, y=0.63, df=dfme.query('env == @env_'), 
                 coln='err_sens_change', colx = coln_same, xt=xt, alt=alt)
        ttrs0_['env'] = env_
        ttrs0_[coln_same] = xv
        ttrs0 += [ttrs0_]
        if len(ttrssig0):
            row = ttrssig0.iloc[0]
            ttrssig0['env'] = env_
            ttrssig0[coln_same] = xv
            #print(env_,xv,row['pval'],ttstr)
            ttrssigs0 += [ttrssig0]
ttrssigs0 = pd.concat(ttrssigs0, ignore_index=1)
ttrs0 = pd.concat(ttrs0, ignore_index=1)
        
#plotSigAll(ax, 1, 0.1,  df=dfme, lab2tick = )
    
plt.suptitle('Error sensitivity dependence \non error sign consitency',y=1.1)
ax.set_ylabel('Error sensitivity change,\nmean within participant')
ax.set_xlabel('Sign consistency')
renameTickLabels(ax, {'-1':'inconsistent', '1':'consistent' })
#plt.savefig(pjoin(path_fig, f'ESchange_per_sign_consist_{coln_same}.pdf'))

In [ ]:
ttrs0

In [ ]:
# for positive, negative signs of 
display(ttrssigs0)
s0 =''
for i,row in ttrssigs0.query('alt != "two-sided"').iterrows():
    sign = row['ttstr'][-3:]
    sc = None
    if row[coln_same] == 1:
        sc = 'consistent'
    elif row[coln_same] == -1:
        sc = 'inconsistent'
    s = 'In {} environment for {} consecutive error signs ES change mean is {:.2f} (std={:.2f}) {} with p-value = {:.2e}'.\
        format(row['env'],sc,row['err_sens_change_mean'],row['err_sens_change_std'],sign,row['pval'])
    s0 += s + '.\n'
print(s0)

s0 =''
for i,row in ttrs0.query('alt == "two-sided" and pval > 0.05').iterrows():
    sign = row['ttstr'][-3:]
    sc = None
    if row[coln_same] == 1:
        sc = 'consistent'
    elif row[coln_same] == -1:
        sc = 'inconsistent'
    s = 'In {} environment for {} consecutive error signs ES change mean {:.2f} has unclear sign (std={:.2f}) with p-value = {:.2e}'.\
        format(row['env'],sc,row['err_sens_change_mean'],row['err_sens_change_std'],row['pval'])
    s0 += s + '.\n'
print(s0)

In [ ]:
#%debug
from figure.mystatann import square_updiag
from itertools import product
def f(df, coln, colx, colhue, colx_vals= None, colhue_vals=None,
     xts=None):
    
    if colx_vals is None:
        colx_vals = list(sorted(df[colx].unique()))
    if colhue_vals is None:
        colhue_vals = list(sorted(df[colhue].unique()))
    hierpairs = product(colx_vals, colhue_vals)
    
    qss = []
    for a,b in hierpairs:
        if isinstance(b,str):
            b = '"' + b + '"'
        if isinstance(a,str):
            a = '"' + a + '"'
        qs = '{} == {} and {} == {}'.format(colx,a,colhue,b)
        qss += [qs]
        
    #print(qss)
    qs_pairs = square_updiag(qss)
    ttrssig,ttrs = comparePairs(df, coln, [colx,colhue], paired=True, qspairs = qs_pairs)
    return ttrssig,ttrs
ttrssig,_ = f(dfme,'err_sens_change', coln_same, 'env')
ttrssig

In [ ]:
import re    
def parseQs(qs):
    pattern = r"(?P<var1>\w+)\s*==\s*(?P<val1>.*?)\s+and\s+(?P<var2>\w+)\s*==\s*(?P<val2>.*)"
    match = re.search(pattern, qs)
    if match:
        var1 = match.group('var1')
        val1 = match.group('val1')
        var2 = match.group('var2')
        val2 = match.group('val2')
        #print(f"Variable 1: {var1}, Value 1: {val1}")
        #print(f"Variable 2: {var2}, Value 2: {val2}")
        
        return {var1:eval(val1), var2:eval(val2 )}
    print('no match for ',qs)
    return None
#parseQs(s)
# for s in set(ttrssig.qs1.unique()) | set(ttrssig.qs2.unique()) :    
#     print(s)
#     d = parseQs(s)
#     print(d)

sign2nice = {1:'consistent', -1:'inconsistent'}

#ttrssig_ps,_ = comparePairs(me_ps, 'err_sens', 'ps_', alt=['greater'], paired=True, updiag=False)
#display(ttrssig_ps[ttrssig_ps.columns[-5:]])
# within env
s1 = ''
for i,row in ttrssig.query('alternative != "two-sided"').iterrows():
    
    d1 = parseQs( row['qs1'] )
    d2 = parseQs( row['qs2'] )
    sc1 = sign2nice[d1[coln_same]]
    sc2 = sign2nice[d2[coln_same]]
    
    if d1['env'] != d2['env']:
        continue
    s = f"ES change for {d1['env']} environment for {sc1} errors is {row['alternative']} than "
    s += f"ES change for {d2['env']} environment for {sc2} errors is {row['alternative']},"
    #ttstr = row['ttstr']
    #s = ttstr.replace(row['val2'], ps_2nice[row['val2']] ).replace(row['val1'], ps_2nice[row['val1']] )    
    s += f" p-value={row['pval']:.2e}; \n"
    s1 += s
#s = f"Stable > random p-value={row['pval']:.2e}"
print(s1)

#### to be implem

In [ ]:

#qss += []
    
    for qs1,qs2 in qs_pairs:
#     for x1,x2  in pairs:
#     #for i,x1 in enumerate(vals):
#     #    for x2 in vals[i+1:]:
#         try:
#             r = plotSig(ax,x1,x2,ycur,ticklen=ticklen,txt=txt,
#                         hor=hor, df=df, coln=coln, colpair=colpair,
#                         paired=paired,pooled=pooled, verbose=verbose,
#                         alt=alt, meanloc_voffset = meanloc_voffset, graded_signif = graded_signif,
#                         fontsize = fontsize)
#             if len(r):
#                 ycur += yinc
#         except KeyError as e:
#             print(str(e))

    
        if lab2tick is None:
            lab2tick = getLab2Tick(ax, hor)
        #print(lab2tick)

        df_ = df.query(f'{colpair} in [@x1,@x2]')
        assert len(df_)
        ttrssig,ttrs = comparePairs(df_, coln, colpair, paired=paired, alt=alt)
        if (ttrssig is None):
            if verbose:
                display(ttrs)
            #print('no sig')
            return []
        pooled = bool(pooled)
        ttrssig = ttrssig.query('pooled == @pooled and alternative == @alt')
        if len(ttrssig) == 0:
            return []
        assert len(ttrssig) <= 1

        if verbose:
            display(ttrssig)

        # draw hor line connecting
        if not isinstance(x1,str):
            x1 = str(x1)
        if not isinstance(x2,str):
            x2 = str(x2)
        x1t,x2t = lab2tick[x1],lab2tick[x2]
        meanloc = np.mean([x1t,x2t]) + meanloc_voffset
        if hor:
            ax.plot([y-ticklen,y,y,y-ticklen], [x1t,x1t,x2t,x2t], c='k')
        else:
            ax.plot([x1t,x1t,x2t,x2t],
                    [y-ticklen,y,y,y-ticklen], c='k')

        if graded_signif:
            assert txt is None
            txt = ttrssig.iloc[0]['starcode']
            #print(ttrssig)
        #print(x1,x2, y, len(ttrssig))
        #print(meanloc)
        #meanloc = 0
        if hor:
            ax.text(y,meanloc,txt)
        else:
            ax.text(meanloc,y,txt, ha='center', fontsize = fontsize)
        return ttrssig

    
    
    #return list(pairs)
f(dfme, coln_same, 'env')

## ES itself

In [ ]:
import warnings
from figure import renameTickLabels
dfcs2  = truncateDf(dfcs_, 'err_sens', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env'])

coln_same = 'err_sign_same3'
qs = f'{coln_same} != 0.'
grp = dfcs2.query(qs).groupby(['subject',coln_same,'env'],
                             observed=True)
dfme = grp.mean(numeric_only=1).reset_index()
dfme['Environment'] = dfme['env']

with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    warnings.filterwarnings('ignore',category=UserWarning)
    fg = sns.catplot(data=dfme, kind='box',
         y ='err_sens', x=coln_same,
         hue='Environment',hue_order=['stable','random'],
         palette=['tab:orange','tab:grey'],
        showfliers=True, legend_out=False)
#fg.apply(lambda ax: ax.axline(y=0))
#def f(data, **kwargs):
#    plt.axhline(y=0)
#fg.map(f, ls=":", c=".5" )
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',c='r')
    
plt.suptitle('Error sensitivity dependence \non error sign consitency',y=1.1)
ax.set_ylabel('Error sensitivity, \nmean within participant')
ax.set_xlabel('Sign consistency')
renameTickLabels(ax, {'-1':'inconsistent', '1':'consistent' })
#plt.savefig(pjoin(path_fig, 'ESchange_per_sign_consist.pdf'))

# Generalization (Fig 4)

In [ ]:
# Need to re-read entire dataset and don't limit to just one columns calc

In [ ]:
# already truncated
#dfc_multi_tsz = dfall_mshsz#.query('thr == "mestd*0"')
dfc_multi_tsz = dfall_mshsz.query('retention_factor_s == "0.924"')

In [ ]:
print('min,max ES {:.3f}, {:.3f}'.format( dfc_multi_tsz.err_sens.min(), dfc_multi_tsz.err_sens.max() ) )
assert dfc_multi_tsz['retention_factor_s'].nunique() == 1

In [ ]:
(dfc_multi_tsz['dist_rad_from_prevtgt_shiftrespect'] == dfc_multi_tsz['dist_rad_from_prevtgt2']).sum()

In [ ]:
df_ = dfc_multi_tsz.query('trial_shift_size > 1 and trial_group_col_calc == "trialwe"')
neq = (df_['dist_rad_from_prevtgt_shiftrespect'] != df_['dist_rad_from_prevtgt2']).sum()
eq = (df_['dist_rad_from_prevtgt_shiftrespect'] == df_['dist_rad_from_prevtgt2']).sum()
print( neq/len(df_), eq/len(df_))

In [ ]:
(df_[['subject_ind','trial_index','trial_shift_size','target_locs',
    'dist_rad_from_prevtgt_shiftrespect','dist_rad_from_prevtgt2']]).iloc[12:]

In [ ]:
#distcol = 'dist_rad_from_prevtgt2'
distcol = 'dist_rad_from_prevtgt_shiftrespect'

drs = dfc_multi_tsz[distcol].unique();
drs_f = np.array( list(map( float, drs) ) )
drs = list(sorted(drs,key = lambda x: float(x)))


#dfc_s = dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']
# TODO: should I restrict to one env?
dfc_s = dfc_multi_tsz.query('trial_group_col_calc == "trialwe"')
grp = dfc_s.groupby(['trial_shift_size', distcol])
dts = dfc_multi_tsz['trial_shift_size'].unique()
nt = len(dts)

print(drs, dts)

assert nt == 4, nt
assert len(drs) == 4, len(drs)

In [ ]:
drpairs = [(drs[i],drs[i+1] ) for i in range(3)]
drpairs += [(drs[0],drs[i] ) for i in range(2,4)]
drpairs += [(drs[1],drs[3] )]
print(drpairs)

dtpairs = [(dts[i],dts[i+1] ) for i in range(3)]
dtpairs += [(dts[0],dts[i] ) for i in range(2,4)]
dtpairs += [(dts[1],dts[3] )]
dtpairs

In [ ]:
def getPvals(dftmp,fitcol,pairs):
    from scipy.stats import ttest_ind
    pvalues = []
    for drp in pairs:
        if isinstance(drp[0],str):
            vs1 = dftmp.query(f'{fitcol} == "{drp[0]}"')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == "{drp[1]}"')['err_sens']
        else:
            vs1 = dftmp.query(f'{fitcol} == {drp[0]}')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == {drp[1]}')['err_sens']
        #ttr = ttest_ind(vs1,vs2)
        ttr = ttest_ind(vs1,vs2, alternative='greater')
        pvalues += [ttr.pvalue]
        
        print(drp, ttr.pvalue)

    formatted_pvalues = [f'p={pvalue:.2e}' for pvalue in pvalues]
    return pvalues, formatted_pvalues

# make polynomial fits
def plotPolys(ax, dftmp, fitcol, degs=range(2,6), mean=1):
    if mean:
        me = dftmp.groupby(fitcol).median(numeric_only=1).reset_index()
        dftmp = me
    dftmp[fitcol] = pd.to_numeric(dftmp[fitcol] )
    esv, dv = dftmp[['err_sens',fitcol]]._values.T
    print(np.min(dv),dv,dv-np.min(dv),esv)
    #pr = np.polyfit(esv,dv,2)
    from numpy.linalg import LinAlgError
    dvu = np.unique(dv)
    dvu = np.array( list(sorted(dvu)) )
    print(dvu)
    for deg in degs:
        try:
            pr = np.polyfit(dv-np.min(dv),esv-np.min(esv),deg)        
        except (SystemError,LinAlgError):
            print(f'Failed deg={deg}')
            print(dv,esv, np.std(dv))
            continue
            
        poly = np.poly1d(pr)
        #if len(degs) > 1:
        if mean:
            lbl = f'polynomial fit of means deg={deg}'
        else:
            lbl = f'polynomial fit deg={deg}'
        #else:
        #    lbl = None
        esv2 = poly(dvu-np.min(dvu)) + np.min(esv)
        print(dvu-np.min(dvu), esv)
        ax.plot(range(len(dvu)) , poly(dvu-np.min(dvu)) + np.min(esv), 
                label=lbl, c='grey', lw=0.85 )
    return pr
    #ax.legend(loc='lower right')

#plotPolys(ax,dftmp,fitcol,mean=0)
#ax = plt.gca()
#plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)

In [ ]:
dfc_multi_tsz.groupby(['subject','environment',distcol]).size()

In [ ]:
from figure.mystatann import plotSigAll
# not pooled
#%debug

hue = 'env'
env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-2,3]
meanfit=1
qs_spatial = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'
qs_temporal = f'trial_group_col_calc == "trialwe" and {distcol} == "0.00"'
#bnqs = [  ('grid_line_violin_spatial','Spatial distance [deg]',drs,drpairs,'dist_rad_from_prevtgt2', f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'),
#        ( 'grid_line_violin_temporal','Temporal distance [number of trials]',dts,dtpairs,'trial_shift_size','trial_group_col_calc == "trialwe" and dist_rad_from_prevtgt2 == "0.00"')]
bnqs = {'grid_line_violin_spatial': ('Spatial distance [deg]',drs,drpairs,
            distcol, qs_spatial),
        'grid_line_violin_temporal':('Temporal distance [number of trials]',dts,dtpairs,
            'trial_shift_size',qs_temporal)}

dfr = []
for env_type in ['stable','random']:
    color = env2color[env_type]
#    for bn,xlab,order,pairs,fitcol,qs in bnqs:
    for bn,(xlab,order,pairs,fitcol,qs) in bnqs.items():
        #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
        #qs = qs + ' and trial_shift_size == 1'
        if env_type in ['stable', 'random']:
            qs += ' and env == @env_type'
            hue = None
            
        dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','environment',fitcol])
        dftmp0 = dftmp0.mean(numeric_only = True).reset_index()                
        dfr += [dftmp0]

        #dftmp0 = dfc_multi_tsz.query(qs)
        #fitcol = 'dist_rad_from_prevtgt2'
        fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                         hue=hue, x=fitcol, order = order,
                          color=color, aspect=aspect, cut=True)
        #dftmp = dfc_multi_tsz.query(qs).copy()
        dftmp = dftmp0.copy()
        #pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)

        ax = fg.axes.flatten()[0]
        ttrs,ttrs_sig = plotSigAll(ax, 1.3, 0.2, 0.05, df=dftmp, 
                   coln = 'err_sens', colpair = fitcol,
                  verbose = 0)

        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',c='r')
            ax.set_ylabel('Error sensitivity', fontdict={'fontsize':12})
            ax.set_xlabel(xlab, fontdict={'fontsize':12} )
            if bn == 'grid_line_violin_spatial':
                ax.set_xticklabels(['same target', 45, 90, 135])
            ax.set_ylim(ylim)
            ax.set_title(f'{env_type} environment', y = 1,
                        fontdict={'fontsize':15, 'fontweight':'bold'})


#         from statannotations.Annotator import Annotator    
#         annotator = Annotator(ax,pairs, data=dftmp0, x=fitcol, y='err_sens',
#                              plot='violinplot', order=order)
#         #annotator.set_custom_annotations(formatted_pvalues)
#         annotator.set_pvalues(pvalues)
#         annotator.annotate()

        ax.set_ylim(-1,3)

        plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
        #bn = 'grid_line_violin_spatial'
        plt.savefig(pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.svg'))
        plt.savefig(pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.pdf'))

In [ ]:
#corrMean?
#compare0?
import gc; gc.collect()

In [ ]:
# compute mean pearson corr between temporal/spatial distance and err sens
dfr = []
for bn,(xlab,order,pairs,fitcol,qs) in bnqs.items():
    dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','env',fitcol])
    dftmp0 = dftmp0.mean(numeric_only = True).reset_index()
    #dftmp0['thr'] = "_"
    dftmp0[fitcol] = dftmp0[fitcol].astype(float)
    corrme,corrme_sep = corrMean(dftmp0, coltocorr=fitcol, 
        stagecol='env' , coln='err_sens', method='spearman')
    corrme_sep['antype'] = bn
    dfr += [corrme_sep.reset_index()]
dfr = pd.concat(dfr, ignore_index=True)

In [ ]:
dfr['method']

In [ ]:
ttrss = []
for antype in bnqs.keys():
    for env in dfr.env.unique():
        ttrs = compare0(dfr.query('antype == @antype and env == @env'), 'r', ['less'], 'r')
        ttrs['antype'] = antype
        ttrs['env'] = env
        ttrss += [ttrs]
ttrs = pd.concat(ttrss, ignore_index=True)
assert len(ttrs.query('pval <= 0.05')) == 4

In [ ]:
# slope sign
s0 = ''
for i,row in ttrs.iterrows():
    ant = row['antype'].split('_')[-1]        
    #row['alt']
    ttsshrt = row['ttstr'][-3:]
    s = 'for {} environment the the mean pearson correlation of ES versus {} distance is {:.2f} (std={:.2f}) {}, p-value = {:.2e}'.format(
        row['env'], ant, row['r_mean'], row['r_std'], ttsshrt, row['pval']
    )
    #print(s)
    s0 += s[0].upper() + s[1:] + '. \n'
print(s0)

In [ ]:
#dfr.query('antype == @antype')

In [ ]:
ttrss = []
for antype in bnqs.keys():    
    ttrssig,ttrs = comparePairs(dfr.query('antype == @antype'), 'r', 'env', 
            alt = ['greater','less','two-sided'], 
                        paired=True, pooled=0 )
    ttrs['antype'] = antype
    #ttrs['env'] = env
    ttrss += [ttrs]
ttrs = pd.concat(ttrss, ignore_index=True)
#assert len(ttrs.query('pval <= 0.05')) == 4

In [ ]:
display(ttrs)
ttrssig = ttrs.query('pval <= 0.05 and alternative == "less"')
print('sig only')
display(ttrssig)

In [ ]:
ttrs.query('alternative == "two-sided" and antype == "grid_line_violin_temporal"')

In [ ]:
#ttrs_sig
if len(ttrssig):
    print('!! SOME SIG !!!')
    pv = ttrssig.iloc[0]['pval']
else:
    pv = ttrs.query('alternative == "two-sided" and antype == "grid_line_violin_temporal"').iloc[0]['pval']
s_ = '{:.2e}'.format( pv )
pvtemp = pv

pv = ttrs.query('alternative == "two-sided" and antype == "grid_line_violin_spatial"').iloc[0]['pval']
pvspat = pv
s_2 = '{:.2e}'.format( pv )
if len(ttrssig):
    s = f'Temporal slope in random is stronger negative than in stable, p-value={s_}. '
else:
    s = f'Temporal slopes in random and stable are not different, p-value={s_}. '
s += f'Spatial slopes in random and stable are not different, p-value={s_2}. '
#print(s)


print(s0,'\n',s)

In [ ]:
print(f'{pvspat:.2e}, {pvtemp:.2e}')

## new stats (D. Herzfeld -suggetsed)

In [ ]:
dfc_multi_tsz.query(qs).groupby(['subject','env',fitcol,'trial_index']).size().max()

### take all trials

In [ ]:
qs = 'trial_group_col_calc == "trialwe"'# and dist_rad_from_prevtgt2 == "0.00"'
df_ = dfc_multi_tsz.query(qs)
print(len(df_))

In [ ]:
grp_shifts = df_.groupby(['subject','env','trial_index'])
grp_shifts.size().min(), grp_shifts.size().mean(), grp_shifts.size().max()

In [ ]:
# super long
method = 'spearman' # 'pearson'
def f(subdf):
    r = pg.corr( subdf['trial_shift_size'], subdf['err_sens'], method=method)
    r['method'] = method
    return r
corrs_shifts = grp_shifts.apply(f)

In [ ]:
corrs_shifts.to_pickle(pjoin(path_data, f'corrs_shifts_{method}.pkl'))

In [ ]:
r_per_subj = corrs_shifts.reset_index().groupby(['env','subject'])['r'].mean().reset_index()
r_per_subj

I have tried to implement what D. Herzfeld suggested (at least as I understood him): to calculate statistics for temporal generalization differently. For every environment for every subject for every trial I have taken the four numbers (corresponding to 4 temporal shifts used to compute ES) and computes correlations between these four numbers and the shift size. Then I averaged the spearman rho across trial indices and compared the result with zero within environment (so 20 numbers for each environment).

It gave me a bit puzzling result at first sight not consistent with the generalization picture (Fig 4 of the manuscript): for random, as previously, the rho is significantly negative. But for stable it is significantly positive!

Probably it is the case because the error mostly shrinks in time in stable environment and thus the quantity
   correction_{n-1} / e_{n-k}
grows

How do you think, shall we report it in results? Or just leave for discussion (or even supplementary)? It might be about confusing for the reader. Also, I have not yet managed to find where this method was used before (it seems not to be there in the reference that Herzfeld gave us during the call. Or maybe it requires deeper reading of that paper).

In [ ]:
for env,alt in zip(['stable','random'],['greater','less']):
    print(env)
    r = compare0(r_per_subj.query('env == @env'), 'r', alt, ['r'])
    display(r)

### take only same target trials

In [ ]:
print(distcol)

In [ ]:
qs = f'trial_group_col_calc == "trialwe" and {distcol} == "0.00"'

In [ ]:
df_ = dfc_multi_tsz.query(qs)
print(len(df_))
grp_shifts = df_.groupby(['subject','env','trial_index'])
grp_shifts.size().min(), grp_shifts.size().mean(), grp_shifts.size().max()

In [ ]:
# super long
method = 'spearman'
def f(subdf):
    r = pg.corr( subdf['trial_shift_size'], subdf['err_sens'], method=method)
    r['method'] = method
    return r
corrs_shifts = grp_shifts.apply(f)

In [ ]:
corrs_shifts

In [ ]:
r_per_subj = corrs_shifts.reset_index().groupby(['env','subject'])[['r','n']].mean().reset_index()
r_per_subj

In [ ]:
for env,alt in zip(['stable','random'],['less','less']):
    print(env)
    r = compare0(r_per_subj.query('env == @env'), 'r', alt, ['r'])
    display(r)

# Break stats

In [ ]:
break_stat = dfall.loc[dfall['pre_break_duration'] > 0].groupby('subject', observed=True) ['pre_break_duration'].describe().mean()
me,std = break_stat['mean'], break_stat['std']
print('Mean pause duration was = {:.1f}s (std = {:.1f}s)'.format(me,std))

# ES vs prev err (supp plot)

In [ ]:
#dfc = df_wthr.copy()
dfc = dfall.copy()
assert dfc.err_sens.max() < 40

# #bins = np.linspace(-1.5,1.5,10)
# bins = np.linspace(-1.5,1.5,10)
# dfc['prev_error_bin'] = pd.cut(dfc['prev_error'], bins)
# grp = dfc.groupby( ['subj','prev_error_bin','env'] )
# dfme = grp['err_sens'].mean().reset_index()
# dfme['prev_error_mid'] = dfme['prev_error_bin'].apply(lambda x: x.mid )
# dfme

In [ ]:
from figure.plots import genStRandLegendHandles
handles = genStRandLegendHandles(rect=False, include_labels=True)

r2d = 180 / np.pi
savefig = 1
bins = np.linspace(-1.2,1.2,15) * r2d; 
#bins = np.linspace(-1.5,1.5,15)  * r2d
#bins = np.linspace(-1.5,1.5,10) * r2d
dfc['prev_error_deg_bin'] = pd.cut(dfc['prev_error'] * r2d, bins) # important to use true error, not pscadj
grp = dfc.groupby( ['subj','prev_error_deg_bin','env','dist_rad_from_prevtgt2'] )
dfme = grp['err_sens'].mean().to_frame()
dfmesz = grp['err_sens'].size().to_frame()
dfme['sz'] = dfmesz['err_sens']
dfme = dfme.reset_index()
dfme['prev_error_deg_mid'] = dfme['prev_error_deg_bin'].apply(lambda x: x.mid )

nbins = len(bins)
minsz = 4
ylim = -0.65,1.1
df_ = dfme.query('sz >= @minsz')

def custom_axis_label(ax, data):
    env = data.env.values[0]
    d = data.dist_rad_from_prevtgt2.values[0]
    ax.set_label_text(f'Dist = {d}')
    
def set_custom_titles(grid, data):
    for ax, (col_val, data_cur_axis) in zip(grid.axes.flat, grid.facet_data()):
        d = data_cur_axis.dist_rad_from_prevtgt2.values[0]
        d = float(d) * r2d
        ax.set_title(f'Distance from\nprevious target = {d:.0f} [deg]')                

#row='env',
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    fg = sns.relplot(kind='line', data=df_, x='prev_error_deg_mid', y='err_sens', 
                      col='dist_rad_from_prevtgt2', hue='env',
                    height=5, hue_order=['stable','random'], palette=['tab:orange','tab:grey'],
                     col_order=['0.00', '0.79', '1.57', '2.36'],
                    legend=False, aspect=0.9)
    #fg = sns.scatterplot(data=dfc.query('env == "stable"'), x='prev_error', y='err_sens', hue='block_name')
    #fg.axhline(0,c='r',ls=':')
    for ax in fg.axes.flatten():
        ax.axhline(0,c='r',ls=':')
        ax.axvline(0,c='r',ls=':')
        ax.set_ylabel('error sensitivity')
        ax.set_xlabel('previous error [deg]')
        ax.set_ylim(ylim)
        
    set_custom_titles(fg, df_)
    
    ax.legend(handles=handles, facecolor='white',                                                                                                                                                                                                       
    loc='lower right', framealpha=0.75, fontsize='medium')  

        #ax.set_ylim(-15,15)
    #addTitleInfo(ax)
    
    plt.suptitle('ES dependence on previous error')
    plt.tight_layout()
    
    print('minsz = ',minsz)
    if savefig:
        plt.savefig(pjoin(path_fig, f'ES_per_prev_error_binned_inctgt_minsz={minsz}_{nbins}.pdf'))
    else:
        print('Skipping fig saving')

# Additional stats (supp)

In [ ]:
mt = 0.754
st = 0.190
s = f'Mean movement time = {mt:.2f}s (std = {st:.2f}s)'
print(s)

In [ ]:
dfall.groupby('subject')['trial_duration'].mean().describe()

In [ ]:
dfall.groupby('subject')['movement_duration'].mean().describe()

In [ ]:
from behav_proc import comparePairs    
df_ = dfall.query('thr == "mestd*0" and env == "random"')
varn = 'err_sens'
col = 'block_name'

print('Compare different random blocks')
df_ = dfall.query('env == "random"')
varn = 'err_sens'
col = 'block_name'

ttrssig, ttrs = comparePairs(df_,varn,col)
if ttrssig is None:
    print('None')
else:
    ttrssig[statcols_toshow]
ttrs

In [ ]:
#'val1','val2',
statcols_toshow = ['varn','T','pval','val1_nice','val2_nice','alternative','pooled','ttstr', 'ttstr_nice','dof']

In [ ]:
stages = ['pre1','pert1_1','washout1_1','pert1_2','washout1_2'] + ['pre2','pert2_1','washout2_1','pert2_2','washout2_2']
ind2stage = dict(zip(np.arange(len(stages)),stages))
ind2stage[-1] = 'random'
ind2stage

def nicener(s):
    # we have to start from -1 otherwise bad
    inds = [-1] + list(range(10))
    for i in inds:
        ps = ind2stage[i]
        
        si = str(i)
        sil = len(si)
        if s.startswith(si + ' '):# or s.startswith(si + '>'):
            s = ps + s[sil:]
        elif s.endswith(' ' + si):# or s.endswith('>' + si):
            s = s[:-sil] + ps
        
    return s
ttrssig['ttstr_nice'] = ttrssig['ttstr'].apply(nicener)
ttrssig['ttstr_nice']

In [ ]:
me = dfall.groupby(['thr','subject','pert_stage']).\
    mean(numeric_only=1).reset_index()
me_pert_stage = me

for col, me in [('pert_stage',me_pert_stage)]:#[('env', me_env), ('ps_', me_ps)]:
    for qs in ['thr == "mestd*0"']:
        df_ = dfall #me.query(qs)
        print(col, qs)
        varn = 'err_sens'
        ttrssig, ttrs = comparePairs(df_,varn,col)
        ttrssig['val1_nice'] = ttrssig['val1'].apply(lambda x: ind2stage[x])
        ttrssig['val2_nice'] = ttrssig['val2'].apply(lambda x: ind2stage[x])
        ttrssig['ttstr_nice'] = ttrssig['ttstr'].apply(nicener)
        
        display(ttrssig.query('alternative != "two-sided"')\
                [statcols_toshow])

In [ ]:
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1")')[['ttstr_nice','pval']] )

In [ ]:
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1") and val1_nice.str.contains("pert") and val2_nice.str.contains("pert")')[['ttstr_nice','pval']] )
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1") and val1_nice.str.contains("wash") and val2_nice.str.contains("wash")')[['ttstr_nice','pval']] )
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1") and val1_nice.str.contains("pre") and val2_nice.str.contains("pre")')[['ttstr_nice','pval']] )
print("Last pertrubation has lower ES compared to first and third perturbations (p-values 0.035 and 0.016.")
print("First washout has lower ES compared to the second one (p-value 0.014).")

# within tgt main stats

## no savings 

In [ ]:
corrs_per_subj_me_,corrs_per_subj  = corrMean(dfall_witgt, 
                stagecol = 'pert_stage', coln='err_sens')

# show stat signif
stage_pairs = [(1,6),(3,8)]
ttrs = []
for s1,s2 in stage_pairs:    
    cps1 = corrs_per_subj.reset_index().query('pert_stage == @s1').set_index(['subject'])
    cps2 = corrs_per_subj.reset_index().query('pert_stage == @s2').set_index(['subject'])
    cps_dif = cps1['r'] - cps2['r'] 
    cps_dif= cps_dif.to_frame()        
    ttr = compare0(cps_dif, 'r', alt='two-sided')
    ttr['stage_pair'] = f'{s1}-{s2}'
    ttrs += [ttr]
ttrs = pd.concat(ttrs)
display( ttrs.query('pval <= 1e-2') )

stage_pairs_nice = {"1-6":'first and last', "3-8":'second and third'}

display(ttrs)
print('\n\nNo savings:')
for irow,row in ttrs.iterrows():
    sp = row['stage_pair']
    pv=row['pval']

    #print(sp,pv)
    print('ES during {} perturbations are not significantly different, p-value = {:.2e}.'.\
              format(stage_pairs_nice[sp],pv) )

## Fig 2 CD

In [ ]:
## plots

In [ ]:
#me.subject.nunique()

In [ ]:
from figure import renameTickLabels, palette_stabrand

thr = "mestd*0"
me = dfall_witgt.groupby(['thr','subject','env'], observed=True).\
    mean(numeric_only=1).reset_index()
me_env = me
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), 
                 kind='violin', y='err_sens', 
    x='env', col='thr', order=['stable','random'],
                palette = palette_stabrand)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
from figure.mystatann import plotSigAll
ylast, ttrssig_env = plotSigAll(ax, 0.83, 0.05, ticklen=0.02,
       df=me, coln='err_sens', colpair = 'env')
    
ax.annotate('C', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')
fg.set_ylabels('Error sensitivity')
ax.set_xlabel('Environment')

fign = 'Fig2C'
plt.savefig(pjoin(path_fig,fign + '.svg'))
plt.savefig(pjoin(path_fig,fign + '.pdf'))
plt.show()
    
###################
# non-pooled stages and env comparison
me = dfall_witgt.groupby(['thr','subject','ps2_'], observed=True).\
    mean(numeric_only=1).reset_index()
me_ps = me
#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), kind='violin', y='err_sens', 
    x='ps2_', col='thr', order=['pre','pert','washout','rnd'],
                hue = 'ps2_')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
#addTitleInfo(fg.axes.flatten()[0])
#plt.gcf().add_subplot_labels(['C'])

ylast, ttrssig_ps = plotSigAll(ax, 2.05, 0.14, ticklen=0.05,
       df=me, coln='err_sens', colpair = 'ps2_', fontsize = 10)

ax.annotate('D', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

ax.set_xlabel('Experiment stage')
#ax.set_xticklabels(['No perturbation','Perturbation','Washout','Random'], 
#                   rotation=30)
ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )
renameTickLabels(ax, ps_2nice, rotation=30)

fg.set_ylabels('Error sensitivity')
fign = 'Fig2D'
plt.savefig(pjoin(path_fig,fign + '.svg'))
plt.savefig(pjoin(path_fig,fign + '.pdf'))

#fg.set_titles()

In [ ]:
## stats

In [ ]:
# to put in results
ttrs = []
for env in me_env.env.unique():
    r = compare0(me_env.query('env == @env'),'err_sens',cols_addstat=['err_sens'])
    r['env'] = env
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05').set_index('env')
assert len(ttrs_pos) == 2
display(ttrs_pos)

s0 = 'Averages of ES within participant. \n'
for env in me_env.env.unique():
    row = ttrs_pos.loc[env]
    s = (f"{env} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

display(ttrssig_env)
row = ttrssig_env.iloc[0]

s = f"Stable > random p-value={row['pval']:.2e}"
print(s0)
print(s)

In [ ]:
# to put in results
#cocoln = 'ps2_'
ttrs = []
for ps in me_ps.ps2_.unique():
    r = compare0(me_ps.query('ps2_ == @ps'),'err_sens',cols_addstat=['err_sens'])
    r['ps2_'] = ps
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05')
assert len(ttrs_pos) == 4
display(ttrs_pos)
ttrs_pos = ttrs_pos.set_index('ps2_')

s0 = 'Averages of ES within participant: \n'
for ps in me_ps.ps2_.unique():
    row = ttrs_pos.loc[ps]
    s = (f"{ps} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

#display(ttrssig_ps)
#row = ttrssig_ps.iloc[0]

ttrssig_ps,_ = comparePairs(me_ps, 'err_sens', 'ps2_', alt=['greater'], 
                            paired=True, updiag=False)
display(ttrssig_ps[ttrssig_ps.columns[-5:]])
s1 = ''
for i,row in ttrssig_ps.iterrows():
    ttstr = row['ttstr']
    s = ttstr.replace(row['val2'], ps_2nice[row['val2']] ).replace(row['val1'],
                                                                   ps_2nice[row['val1']] )    
    s += f" p-value={row['pval']:.2e}; \n"
    s1 += s
#s = f"Stable > random p-value={row['pval']:.2e}"
print(s0)
print(s1)

In [ ]:
# show that dif between pert and rand is not there
psvals = ["pert","rnd"]
ttrssig,ttrssig_all = comparePairs(me_ps.query('ps2_.isin(@psvals)'),
                         'err_sens', 'ps2_', alt=['two-sided'], 
                        paired=True, updiag=True, pooled=0)
display(ttrssig_all)
assert len(ttrssig_all) == 1
row = ttrssig_all.iloc[0]
if row['pval'] > 0.05:    
    s = 'The difference between ES in '
    s += '{} and {} conditions is not significant ('.format( 
        ps_2nice[row['val2']], ps_2nice[row['val1'] ] )
    s += f" p-value={row['pval']:.2e})."
    #s1 += s
    print(s)
else:
    print('Actually there is significance')

# New stats for err

In [ ]:
warnings.filterwarnings("ignore",category=UserWarning) 

In [ ]:
ttrssig,ttrs = comparePairs(dfall,'error_pscadj','block_name')

In [ ]:
ttrssig.query('alternative != "two-sided"')[['pval','ttstr','pooled']]

In [ ]:
me = dfall.groupby(['subject','env','block_name'])['error_pscadj'].mean().to_frame().reset_index()

In [ ]:
me

In [ ]:
from behav_proc import compare0
for bn in dfall.block_name.unique():
    df_ = dfall.query('block_name == @bn')
    ttrs = compare0(df_,'error_pscadj').query('pval <= 0.05')
    print(bn)
    display(ttrs)

In [ ]:
fg = sns.catplot(data = dfall,kind='violin',y='error_pscadj',
           x='block_name', hue='env')
fg.refline(y=0)

fg = sns.catplot(data = me,kind='violin',y='error_pscadj',
           x='block_name', hue='env')
fg.refline(y=0)